In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Part I. ETL Pipeline for Pre-Processing the Files"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### Import Python packages "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import Python packages \n",
    "import pandas as pd\n",
    "import cassandra\n",
    "import re\n",
    "import os\n",
    "import glob\n",
    "import numpy as np\n",
    "import json\n",
    "import csv"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### Creating list of filepaths to process original event csv data files"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "/workspace/home\n"
     ]
    }
   ],
   "source": [
    "# checking your current working directory\n",
    "print(os.getcwd())\n",
    "\n",
    "# Get your current folder and subfolder event data\n",
    "filepath = os.getcwd() + '/event_data'\n",
    "\n",
    "# Create a for loop to create a list of files and collect each filepath\n",
    "for root, dirs, files in os.walk(filepath):\n",
    "    \n",
    "# join the file path and roots with the subdirectories using glob\n",
    "    file_path_list = glob.glob(os.path.join(root,'*'))\n",
    "    #print(file_path_list)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### Processing the files to create the data file csv that will be used for Apache Casssandra tables"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [],
   "source": [
    "# initiating an empty list of rows that will be generated from each file\n",
    "full_data_rows_list = [] \n",
    "    \n",
    "# for every filepath in the file path list \n",
    "for f in file_path_list:\n",
    "\n",
    "# reading csv file \n",
    "    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: \n",
    "        # creating a csv reader object \n",
    "        csvreader = csv.reader(csvfile) \n",
    "        next(csvreader)\n",
    "        \n",
    " # extracting each data row one by one and append it        \n",
    "        for line in csvreader:\n",
    "            #print(line)\n",
    "            full_data_rows_list.append(line) \n",
    "            \n",
    "# uncomment the code below if you would like to get total number of rows \n",
    "#print(len(full_data_rows_list))\n",
    "# uncomment the code below if you would like to check to see what the list of event data rows will look like\n",
    "#print(full_data_rows_list)\n",
    "\n",
    "# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \\\n",
    "# Apache Cassandra tables\n",
    "csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)\n",
    "\n",
    "with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:\n",
    "    writer = csv.writer(f, dialect='myDialect')\n",
    "    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\\\n",
    "                'level','location','sessionId','song','userId'])\n",
    "    for row in full_data_rows_list:\n",
    "        if (row[0] == ''):\n",
    "            continue\n",
    "        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "6821\n"
     ]
    }
   ],
   "source": [
    "# check the number of rows in your csv file\n",
    "with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:\n",
    "    print(sum(1 for line in f))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Begin writing your Apache Cassandra code in the cells below"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### Creating a Cluster"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [],
   "source": [
    "# This should make a connection to a Cassandra instance your local machine \n",
    "# (127.0.0.1)\n",
    "\n",
    "from cassandra.cluster import Cluster\n",
    "cluster = Cluster()\n",
    "\n",
    "# To establish connection and begin executing queries, need a session\n",
    "session = cluster.connect()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Part II. Complete the Apache Cassandra coding portion of your project. \n",
    "\n",
    "## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: \n",
    "- artist \n",
    "- firstName of user\n",
    "- gender of user\n",
    "- item number in session\n",
    "- last name of user\n",
    "- length of the song\n",
    "- level (paid or free song)\n",
    "- location of the user\n",
    "- sessionId\n",
    "- song title\n",
    "- userId\n",
    "\n",
    "The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>\n",
    "\n",
    "<img src=\"images/image_event_datafile_new.jpg\">"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### Create Keyspace"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {},
   "outputs": [],
   "source": [
    "# TO-DO: Create a Keyspace \n",
    "try:\n",
    "    session.execute(\"\"\"\n",
    "    CREATE KEYSPACE IF NOT EXISTS udacity \n",
    "    WITH REPLICATION = \n",
    "    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }\"\"\"\n",
    ")\n",
    "\n",
    "except Exception as e:\n",
    "    print(e)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### Set Keyspace"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [],
   "source": [
    "# TO-DO: Set KEYSPACE to the keyspace specified above\n",
    "try:\n",
    "    session.set_keyspace('udacity')\n",
    "except Exception as e:\n",
    "    print(e)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Create queries to ask the following three questions of the data\n",
    "\n",
    "### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4\n",
    "\n",
    "\n",
    "### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182\n",
    "    \n",
    "\n",
    "### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'\n",
    "\n",
    "\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 27,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Table 'music_history' has been created successfully.\n"
     ]
    }
   ],
   "source": [
    "query = \"\"\"\n",
    "CREATE TABLE IF NOT EXISTS music_history (\n",
    "    user_id TEXT,\n",
    "    session_id TEXT,\n",
    "    item_in_session TEXT,\n",
    "    artist TEXT,\n",
    "    song TEXT,\n",
    "    first_name TEXT,\n",
    "    last_name TEXT,\n",
    "    gender TEXT,\n",
    "    length TEXT,\n",
    "    level TEXT,\n",
    "    location TEXT,\n",
    "    PRIMARY KEY (user_id, session_id)\n",
    ");\n",
    "\"\"\"\n",
    "\n",
    "# Execute the query to create the table\n",
    "try:\n",
    "    session.execute(query)\n",
    "    print(\"Table 'music_history' has been created successfully.\")\n",
    "except Exception as e:\n",
    "    print(f\"An error occurred while trying to create the table: {e}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 28,
   "metadata": {
    "scrolled": false
   },
   "outputs": [],
   "source": [
    "# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#\n",
    "file = 'event_datafile_new.csv'\n",
    "\n",
    "with open(file, encoding = 'utf8') as f:\n",
    "    csvreader = csv.reader(f)\n",
    "    next(csvreader) # skip header\n",
    "    for line in csvreader:\n",
    "## TO-DO: Assign the INSERT statements into the `query` variable\n",
    "        query = \"INSERT INTO music_history (artist, first_name, gender, item_in_session, last_name, length, level, location, session_id, song, user_id)\"\t\t\t\n",
    "        query = query + \" VALUES (%s, %s, %s, %s,%s, %s, %s, %s,%s, %s, %s)\"\n",
    "        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.\n",
    "        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`\n",
    "        session.execute(query, (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10]))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "View 'music_history_by_session_view' has been created successfully.\n"
     ]
    }
   ],
   "source": [
    "query = \"\"\"\n",
    "CREATE MATERIALIZED VIEW music_history_by_session_view AS\n",
    "SELECT *\n",
    "FROM music_history\n",
    "WHERE session_id IS NOT NULL AND item_in_session IS NOT NULL\n",
    "PRIMARY KEY ((session_id, item_in_session), user_id);\n",
    "\"\"\"\n",
    "try:\n",
    "    session.execute(query)\n",
    "    print(\"View 'music_history_by_session_view' has been created successfully.\")\n",
    "except Exception as e:\n",
    "    print(f\"An error occurred while trying to create the table: {e}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Faithless Music Matters (Mark Knight Dub) 495.3073\n"
     ]
    }
   ],
   "source": [
    "query = \"SELECT artist, song, length FROM music_history_by_session_view WHERE session_id='338' AND item_in_session='4'\"\n",
    "try:\n",
    "    rows = session.execute(query)\n",
    "except Exception as e:\n",
    "    print(e)\n",
    "    \n",
    "for row in rows:\n",
    "    print (row.artist, row.song, row.length)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### Do a SELECT to verify that the data have been inserted into each table"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "metadata": {
    "scrolled": true
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Jordan Rudess Cecilia F 1 Owens 1367.84934 free Atlanta-Sandy Springs-Roswell, GA 225 Tarkus 6\n",
      "Ron Van Den Beuken Cecilia F 0 Owens 506.5922 free Atlanta-Sandy Springs-Roswell, GA 229 Timeless 6\n",
      "Motionless in White Cecilia F 0 Owens 217.57342 free Atlanta-Sandy Springs-Roswell, GA 288 Ghost In The Mirror 6\n",
      "Phil Collins Cecilia F 0 Owens 239.85587 free Atlanta-Sandy Springs-Roswell, GA 406 The Way You Look Tonight (Live Version) 6\n",
      "Tub Ring Cecilia F 0 Owens 233.69098 free Atlanta-Sandy Springs-Roswell, GA 423 Invalid 6\n",
      "Asia 2001 Cecilia F 1 Owens 150.30812 free Atlanta-Sandy Springs-Roswell, GA 444 Epilogue 6\n",
      "The New Pornographers Cecilia F 1 Owens 173.94893 free Atlanta-Sandy Springs-Roswell, GA 453 Falling Through Your Clothes 6\n",
      "A Hope For Home Cecilia F 1 Owens 388.38812 free Atlanta-Sandy Springs-Roswell, GA 5 Absolution: Of Flight and Failure 6\n",
      "Lykke Li Cecilia F 1 Owens 232.35873 free Atlanta-Sandy Springs-Roswell, GA 535 Melodies & Desires 6\n",
      "Justin Bieber Cecilia F 1 Owens 222.92853 free Atlanta-Sandy Springs-Roswell, GA 735 Stuck In The Moment 6\n",
      "Line Renaud Cecilia F 2 Owens 176.16934 free Atlanta-Sandy Springs-Roswell, GA 763 Le Soir 6\n",
      "Dwight Yoakam Cecilia F 2 Owens 239.3073 free Atlanta-Sandy Springs-Roswell, GA 847 You're The One 6\n",
      "The Carter Family Cecilia F 1 Owens 171.54567 free Atlanta-Sandy Springs-Roswell, GA 870 Little Log Cabin By The Sea 6\n",
      "Toto Cecilia F 1 Owens 411.19302 free Atlanta-Sandy Springs-Roswell, GA 948 Home Of The Brave 6\n",
      "Thousand Foot Krutch Andrea F 0 Butler 58.87955 free Pensacola-Ferry Pass-Brent, FL 148 The Invitation 90\n",
      "Roky Erickson Andrea F 1 Butler 248.0322 free Pensacola-Ferry Pass-Brent, FL 745 Cold night for alligators 90\n",
      "Charttraxx Karaoke Martin M 5 Johnson 225.17506 free Minneapolis-St. Paul-Bloomington, MN-WI 1008 Fireflies 55\n",
      "AFI Martin M 1 Johnson 190.45832 free Minneapolis-St. Paul-Bloomington, MN-WI 250 Girl's Not Grey 55\n",
      "Amy Winehouse Martin M 2 Johnson 141.24363 free Minneapolis-St. Paul-Bloomington, MN-WI 331 Some Unholy War 55\n",
      "The Verve Martin M 5 Johnson 360.25424 free Minneapolis-St. Paul-Bloomington, MN-WI 415 Bitter Sweet Symphony 55\n",
      "TRUSTcompany Martin M 0 Johnson 194.2722 free Minneapolis-St. Paul-Bloomington, MN-WI 773 Take It All 55\n",
      "Jadakiss / Swizz Beatz / OJ Da Juiceman Martin M 1 Johnson 191.84281 free Minneapolis-St. Paul-Bloomington, MN-WI 996 Who's Real 55\n",
      "Carla Bruni Sienna F 0 Colon 163.02975 free Las Cruces, NM 159 Quelqu'un M'a Dit (Album Version) 81\n",
      "Vinnie Vincent Invasion Sienna F 0 Colon 303.12444 free Las Cruces, NM 291 Ashes To Ashes (2003 Digital Remaster) 81\n",
      "Metallica Sienna F 0 Colon 415.16363 free Las Cruces, NM 317 Fade To Black 81\n",
      "SOJA Sienna F 2 Colon 285.25669 free Las Cruces, NM 386 Rasta Courage 81\n",
      "Anberlin Rylan M 60 George 348.682 paid Birmingham-Hoover, AL 1076 The Haunting 16\n",
      "Dashboard Confessional Rylan M 5 George 224.54812 free Birmingham-Hoover, AL 15 Ghost Of A Good Thing 16\n",
      "Eminem / Nate Dogg Rylan M 1 George 297.9522 free Birmingham-Hoover, AL 315 'Till I Collapse 16\n",
      "Justin Bieber Rylan M 3 George 212.53179 free Birmingham-Hoover, AL 393 Runaway Love 16\n",
      "Faith No More Rylan M 1 George 326.50404 free Birmingham-Hoover, AL 402 Helpless 16\n",
      "Cascada Rylan M 2 George 184.39791 free Birmingham-Hoover, AL 446 Kids In America 16\n",
      "The Format Rylan M 43 George 225.802 paid Birmingham-Hoover, AL 479 Inches And Falling (I Love_ Love) (Live) 16\n",
      "Band Of Horses Rylan M 18 George 321.14893 paid Birmingham-Hoover, AL 575 The Funeral (Album Version) 16\n",
      "Dr. Dre / Eminem / Alvin Joiner Rylan M 6 George 244.21832 paid Birmingham-Hoover, AL 617 What's The Difference 16\n",
      "Gordon Lightfoot Rylan M 17 George 179.9571 paid Birmingham-Hoover, AL 854 Long Thin Dawn 16\n",
      "Los Wawanco Rylan M 0 George 165.79873 paid Birmingham-Hoover, AL 871 La Casita Blanca 16\n",
      "Haloo Helsinki! Rylan M 72 George 223.42485 paid Birmingham-Hoover, AL 983 Kaaos ei karkaa 16\n",
      "John Mayer Noah M 1 Chavez 250.38322 free Ogden-Clearfield, UT 150 Half Of My Heart 94\n",
      "Pork Dukes Noah M 0 Chavez 133.04118 free Ogden-Clearfield, UT 262 Bend & Flush 94\n",
      "Hoobastank Noah M 1 Chavez 232.17587 free Ogden-Clearfield, UT 492 Born To Lead 94\n",
      "Emmy The Great Noah M 0 Chavez 204.79955 free Ogden-Clearfield, UT 530 Mia 94\n",
      "Amy Winehouse Noah M 3 Chavez 154.14812 free Ogden-Clearfield, UT 755 Love Is A Losing Game 94\n",
      "AFI Hannah F 0 Calhoun 176.48281 free Los Angeles-Long Beach-Anaheim, CA 166 Miseria Cantare--The Beginning 64\n",
      "Journey Hannah F 0 Calhoun 244.06159 free Los Angeles-Long Beach-Anaheim, CA 410 Loved By You 64\n",
      "Cute Is What We Aim For Colm M 8 Santana 172.22485 free Nashville-Davidson--Murfreesboro--Franklin, TN 414 Sweat the Battle Before the Battle Sweats You (Album Version) 67\n",
      "J. Karjalainen & Mustat Lasit Colm M 8 Santana 336.74404 free Nashville-Davidson--Murfreesboro--Franklin, TN 475 Sinisten tÃÂ¤htien alla 67\n",
      "August Burns Red Colm M 4 Santana 252.21179 free Nashville-Davidson--Murfreesboro--Franklin, TN 540 Composure 67\n",
      "BeyoncÃÂ© Colm M 0 Santana 261.74649 free Nashville-Davidson--Murfreesboro--Franklin, TN 599 Halo 67\n",
      "Taylor Swift Colm M 0 Santana 233.89995 free Nashville-Davidson--Murfreesboro--Franklin, TN 66 Love Story 67\n",
      "Kurtis Blow Colm M 0 Santana 353.88036 free Nashville-Davidson--Murfreesboro--Franklin, TN 722 The Breaks 67\n",
      "Audioslave Colm M 0 Santana 295.47057 free Nashville-Davidson--Murfreesboro--Franklin, TN 787 Like A Stone 67\n",
      "Katrina & The Waves Colm M 4 Santana 220.9171 free Nashville-Davidson--Murfreesboro--Franklin, TN 807 Walking On Sunshine 67\n",
      "Gibonni Emily F 2 Benson 379.68934 paid Augusta-Richmond County, GA-SC 1013 OVO MI JE `KOLA 58\n",
      "Buju Banton Emily F 1 Benson 232.48934 paid Augusta-Richmond County, GA-SC 377 Cry No More 58\n",
      "Coldplay Emily F 13 Benson 307.51302 paid Augusta-Richmond County, GA-SC 494 Clocks 58\n",
      "Gang Of Four Emily F 29 Benson 207.25506 paid Augusta-Richmond County, GA-SC 522 Damaged Goods 58\n",
      "Stars Emily F 2 Benson 243.1473 paid Augusta-Richmond County, GA-SC 579 Elevator Love Letter 58\n",
      "DJ Dizzy Emily F 3 Benson 221.1522 paid Augusta-Richmond County, GA-SC 720 Sexy Bitch 58\n",
      "Evergreen Terrace Emily F 2 Benson 172.38159 paid Augusta-Richmond County, GA-SC 768 Zero 58\n",
      "Martin Solveig Emily F 2 Benson 246.282 paid Augusta-Richmond County, GA-SC 821 Something About You 58\n",
      "Gov't Mule Emily F 34 Benson 258.14159 paid Augusta-Richmond County, GA-SC 852 Mr. Man 58\n",
      "Amorphis Emily F 51 Benson 299.07546 paid Augusta-Richmond County, GA-SC 887 Towards And Against 58\n",
      "The Black Keys Lily F 1 Burns 211.01669 free New York-Newark-Jersey City, NY-NJ-PA 1002 Tighten Up 32\n",
      "The Decemberists Lily F 1 Burns 248.55465 free New York-Newark-Jersey City, NY-NJ-PA 1032 Billy Liar 32\n",
      "Madonna Lily F 2 Burns 220.29016 free New York-Newark-Jersey City, NY-NJ-PA 1036 Revolver [feat. Lil Wayne] 32\n",
      "Katie Melua Lily F 1 Burns 218.09587 free New York-Newark-Jersey City, NY-NJ-PA 1045 I Cried For You 32\n",
      "Leftover Crack Lily F 1 Burns 254.98077 free New York-Newark-Jersey City, NY-NJ-PA 1058 Clear Channel (Fuck Off!) 32\n",
      "The Killers Lily F 0 Burns 185.33832 free New York-Newark-Jersey City, NY-NJ-PA 239 I Can't Stay 32\n",
      "The Kingstonians Lily F 0 Burns 172.90404 free New York-Newark-Jersey City, NY-NJ-PA 263 Winey Winey 32\n",
      "Placebo Lily F 0 Burns 324.0224 free New York-Newark-Jersey City, NY-NJ-PA 280 Passive Aggressive 32\n",
      "Justin Bieber Lily F 5 Burns 191.55546 free New York-Newark-Jersey City, NY-NJ-PA 305 Love Me 32\n",
      "Eddie Floyd Lily F 1 Burns 198.19057 free New York-Newark-Jersey City, NY-NJ-PA 31 Guess Who 32\n",
      "The Mantles Lily F 2 Burns 226.53342 free New York-Newark-Jersey City, NY-NJ-PA 349 Don't Lie 32\n",
      "Alphaville Lily F 0 Burns 285.43955 free New York-Newark-Jersey City, NY-NJ-PA 363 Big In Japan (Original) 32\n",
      "Aerosmith Lily F 0 Burns 301.76608 free New York-Newark-Jersey City, NY-NJ-PA 420 Fly Away From Here 32\n",
      "HIM Lily F 1 Burns 212.06159 free New York-Newark-Jersey City, NY-NJ-PA 456 Beautiful 32\n",
      "Jason Mraz Lily F 4 Burns 226.53342 free New York-Newark-Jersey City, NY-NJ-PA 489 After An Afternoon (Eagles Ballroom Live Version) 32\n",
      "The Brecker Brothers Lily F 1 Burns 217.5473 free New York-Newark-Jersey City, NY-NJ-PA 503 East River 32\n",
      "Sisqo Lily F 2 Burns 253.49179 free New York-Newark-Jersey City, NY-NJ-PA 505 Thong Song 32\n",
      "Ray Davies Lily F 1 Burns 285.3873 free New York-Newark-Jersey City, NY-NJ-PA 512 The Tourist 32\n",
      "Kanye West Lily F 1 Burns 238.52363 free New York-Newark-Jersey City, NY-NJ-PA 517 Flashing Lights 32\n",
      "Macaco Lily F 0 Burns 284.52526 free New York-Newark-Jersey City, NY-NJ-PA 546 Mama Tierra 32\n",
      "Eminem / Dr. Dre Lily F 4 Burns 309.83791 free New York-Newark-Jersey City, NY-NJ-PA 554 Say What You Say 32\n",
      "The Notorious B.I.G. Lily F 2 Burns 238.78485 free New York-Newark-Jersey City, NY-NJ-PA 625 The What 32\n",
      "Selena Gomez & The Scene Lily F 2 Burns 184.65914 free New York-Newark-Jersey City, NY-NJ-PA 646 Falling Down 32\n",
      "Tenth Avenue North Lily F 1 Burns 245.34159 free New York-Newark-Jersey City, NY-NJ-PA 711 Love Is Here 32\n",
      "Rihanna Lily F 3 Burns 239.75138 free New York-Newark-Jersey City, NY-NJ-PA 727 G4L 32\n",
      "Samael Lily F 0 Burns 221.33506 free New York-Newark-Jersey City, NY-NJ-PA 756 Under One Flag 32\n",
      "Kid Cudi / MGMT / Ratatat Lily F 0 Burns 295.67955 free New York-Newark-Jersey City, NY-NJ-PA 786 Pursuit Of Happiness (nightmare) 32\n",
      "Slipknot Lily F 1 Burns 159.76444 free New York-Newark-Jersey City, NY-NJ-PA 885 Spit It Out [Explicit] 32\n",
      "Ill NiÃÂ±o Lily F 2 Burns 172.9824 free New York-Newark-Jersey City, NY-NJ-PA 886 Re-Birth 32\n",
      "Combichrist Lily F 0 Burns 336.14322 free New York-Newark-Jersey City, NY-NJ-PA 958 red signal 32\n",
      "Camila Tucker M 1 Garrison 230.81751 free Oxnard-Thousand Oaks-Ventura, CA 555 Abrazame  (Version Acustica) 40\n",
      "Carl Thomas Tucker M 0 Garrison 196.67546 free Oxnard-Thousand Oaks-Ventura, CA 698 You Ain't Right (Album Version) 40\n",
      "Entre RÃÂ­os Tucker M 1 Garrison 281.15546 free Oxnard-Thousand Oaks-Ventura, CA 784 Hoy No 40\n",
      "Daughtry Tucker M 0 Garrison 201.63873 free Oxnard-Thousand Oaks-Ventura, CA 798 Gone 40\n",
      "Tego Calderon Tucker M 0 Garrison 151.74485 free Oxnard-Thousand Oaks-Ventura, CA 874 Pa' Que Retozen (Amended Version) 40\n",
      "The White Stripes Tucker M 1 Garrison 160.31302 free Oxnard-Thousand Oaks-Ventura, CA 976 Blue Orchid 40\n",
      "Amos Lee Ryan M 1 Smith 160.49587 free San Jose-Sunnyvale-Santa Clara, CA 1039 Colors 26\n",
      "Eric B. & Rakim Ryan M 9 Smith 224.23465 free San Jose-Sunnyvale-Santa Clara, CA 1046 Eric B. Is On The Cut 26\n",
      "Symphony X Ryan M 10 Smith 389.45914 free San Jose-Sunnyvale-Santa Clara, CA 1068 Domination 26\n",
      "Betty Wright Ryan M 1 Smith 164.85832 free San Jose-Sunnyvale-Santa Clara, CA 1090 Clean Up Woman (Single/LP Version) 26\n",
      "M.I.A. Ryan M 2 Smith 233.7171 free San Jose-Sunnyvale-Santa Clara, CA 169 Mango Pickle Down River (With The Wilcannia Mob) 26\n",
      "Squeeze Ryan M 5 Smith 229.56363 free San Jose-Sunnyvale-Santa Clara, CA 183 Footprints 26\n",
      "Extrawelt Ryan M 1 Smith 338.23302 free San Jose-Sunnyvale-Santa Clara, CA 232 Stammgast 26\n",
      "Faith No More Ryan M 2 Smith 249.86077 free San Jose-Sunnyvale-Santa Clara, CA 268 Last Cup Of Sorrow 26\n",
      "Creedence Clearwater Revival Ryan M 4 Smith 230.94812 free San Jose-Sunnyvale-Santa Clara, CA 278 It's Just A Thought 26\n",
      "Eliza Doolittle Ryan M 1 Smith 200.30649 free San Jose-Sunnyvale-Santa Clara, CA 322 Police Car 26\n",
      "Shinedown Ryan M 0 Smith 220.39465 free San Jose-Sunnyvale-Santa Clara, CA 346 Second Chance (Album Version) 26\n",
      "Fabolous Ryan M 1 Smith 306.442 free San Jose-Sunnyvale-Santa Clara, CA 356 Can't Deny It [featuring Nate Dogg] [Explicit Version] 26\n",
      "K-OS Ryan M 0 Smith 211.33016 free San Jose-Sunnyvale-Santa Clara, CA 382 EMCEE Murdah 26\n",
      "NOFX Ryan M 0 Smith 65.04444 free San Jose-Sunnyvale-Santa Clara, CA 394 Benny Got Blowed Up 26\n",
      "Okkervil River Ryan M 2 Smith 278.54322 free San Jose-Sunnyvale-Santa Clara, CA 397 Unless It's Kicks 26\n",
      "Sam Cooke Ryan M 3 Smith 122.04363 free San Jose-Sunnyvale-Santa Clara, CA 431 Ain't Misbehavin 26\n",
      "The Living End Ryan M 0 Smith 188.62975 free San Jose-Sunnyvale-Santa Clara, CA 433 Roll On (Album Version) 26\n",
      "Karnivool Ryan M 0 Smith 470.80444 free San Jose-Sunnyvale-Santa Clara, CA 445 Umbra 26\n",
      "Lynyrd Skynyrd Ryan M 0 Smith 216.60689 free San Jose-Sunnyvale-Santa Clara, CA 452 Sweet home Alabama 26\n",
      "Incubus Ryan M 3 Smith 293.38077 free San Jose-Sunnyvale-Santa Clara, CA 472 Black Heart Inertia 26\n",
      "Alicia Keys Ryan M 1 Smith 216.47628 free San Jose-Sunnyvale-Santa Clara, CA 480 Empire State Of Mind (Part II) Broken Down 26\n",
      "Dwight Yoakam Ryan M 1 Smith 239.3073 free San Jose-Sunnyvale-Santa Clara, CA 491 You're The One 26\n",
      "Morningwood Ryan M 2 Smith 176.66567 free San Jose-Sunnyvale-Santa Clara, CA 500 New York Girls 26\n",
      "BeyoncÃÂ© Ryan M 0 Smith 261.74649 free San Jose-Sunnyvale-Santa Clara, CA 529 Halo 26\n",
      "Junior Kimbrough And The Soul Blues Boys Ryan M 1 Smith 350.58893 free San Jose-Sunnyvale-Santa Clara, CA 571 All Night Long 26\n",
      "Train Ryan M 2 Smith 205.45261 free San Jose-Sunnyvale-Santa Clara, CA 583 Marry Me 26\n",
      "Los Campesinos Ryan M 1 Smith 138.10893 free San Jose-Sunnyvale-Santa Clara, CA 607 We throw parties_ you throw knives 26\n",
      "R.E.M. Ryan M 3 Smith 241.162 free San Jose-Sunnyvale-Santa Clara, CA 622 What's The Frequency_ Kenneth? (Radio Version) 26\n",
      "Santana Ryan M 0 Smith 290.71628 free San Jose-Sunnyvale-Santa Clara, CA 644 Daughter Of The Night 26\n",
      "Rufus Wainwright Ryan M 1 Smith 254.17098 free San Jose-Sunnyvale-Santa Clara, CA 657 Waiting For A Dream 26\n",
      "Mariah Carey Ryan M 2 Smith 261.82485 free San Jose-Sunnyvale-Santa Clara, CA 660 Bye Bye 26\n",
      "The xx Ryan M 2 Smith 188.55138 free San Jose-Sunnyvale-Santa Clara, CA 663 Basic Space 26\n",
      "iio Ryan M 0 Smith 270.36689 free San Jose-Sunnyvale-Santa Clara, CA 670 Smooth 26\n",
      "Armin van Buuren Ryan M 1 Smith 435.51302 free San Jose-Sunnyvale-Santa Clara, CA 686 Hold On To Me 26\n",
      "James Blake Ryan M 0 Smith 287.00689 free San Jose-Sunnyvale-Santa Clara, CA 706 Footnotes 26\n",
      "Lady GaGa / Colby O'Donis Ryan M 1 Smith 238.54975 free San Jose-Sunnyvale-Santa Clara, CA 719 Just Dance 26\n",
      "Black Mountain Ryan M 1 Smith 267.41506 free San Jose-Sunnyvale-Santa Clara, CA 725 Stay Free (Album Version) 26\n",
      "Train Ryan M 0 Smith 216.76363 free San Jose-Sunnyvale-Santa Clara, CA 751 Hey_ Soul Sister 26\n",
      "Animal Collective Ryan M 0 Smith 322.0371 free San Jose-Sunnyvale-Santa Clara, CA 767 In The Flowers 26\n",
      "Sean Kingston and Justin Bieber Ryan M 2 Smith 201.9522 free San Jose-Sunnyvale-Santa Clara, CA 783 Eenie Meenie 26\n",
      "Linkin Park Ryan M 1 Smith 208.92689 free San Jose-Sunnyvale-Santa Clara, CA 789 What I've Done (Album Version) 26\n",
      "Evanescence Ryan M 1 Smith 237.11302 free San Jose-Sunnyvale-Santa Clara, CA 794 Bring Me To Life 26\n",
      "ATB Ryan M 3 Smith 191.7122 free San Jose-Sunnyvale-Santa Clara, CA 800 Rising Moon 26\n",
      "A3Bandas Ryan M 1 Smith 107.10159 free San Jose-Sunnyvale-Santa Clara, CA 811 El arca de Noe 26\n",
      "Yonder Mountain String Band Ryan M 2 Smith 152.18893 free San Jose-Sunnyvale-Santa Clara, CA 820 Midwest Gospel Radio 26\n",
      "Guns N' Roses Ryan M 1 Smith 284.05506 free San Jose-Sunnyvale-Santa Clara, CA 845 Don't Cry (Original) 26\n",
      "Genesis Ryan M 0 Smith 265.97832 free San Jose-Sunnyvale-Santa Clara, CA 853 That's All (2007 Digital Remaster) 26\n",
      "The Courteeners Ryan M 1 Smith 225.48853 free San Jose-Sunnyvale-Santa Clara, CA 868 Take Over The World 26\n",
      "Harmonia Ryan M 1 Smith 655.77751 free San Jose-Sunnyvale-Santa Clara, CA 873 Sehr kosmisch 26\n",
      "Obk Ryan M 3 Smith 377.15546 free San Jose-Sunnyvale-Santa Clara, CA 896 Lucifer (The Final Combination Mix) 26\n",
      "the bird and the bee Ryan M 1 Smith 189.51791 free San Jose-Sunnyvale-Santa Clara, CA 899 You're A Cad 26\n",
      "Des'ree Ryan M 0 Smith 196.51873 free San Jose-Sunnyvale-Santa Clara, CA 919 Life 26\n",
      "Jean Michel Jarre Ryan M 2 Smith 461.58322 free San Jose-Sunnyvale-Santa Clara, CA 946 oxygene 2 26\n",
      "Breaks Co-Op Ryan M 1 Smith 199.73179 free San Jose-Sunnyvale-Santa Clara, CA 989 The Otherside (Acoustic Version) 26\n",
      "Angels Of Light & Akron/Family Ryan M 0 Smith 150.33424 free San Jose-Sunnyvale-Santa Clara, CA 991 Awake 26\n",
      "Mike Posner Jahiem M 0 Miles 214.7522 free San Antonio-New Braunfels, TX 228 Cooler Than Me 43\n",
      "Euge Groove Jahiem M 0 Miles 265.482 free San Antonio-New Braunfels, TX 245 Tenderly 43\n",
      "Russell Malone Jahiem M 4 Miles 396.40771 free San Antonio-New Braunfels, TX 265 Heartstrings 43\n",
      "Flying Lotus Jahiem M 0 Miles 115.33016 free San Antonio-New Braunfels, TX 369 Bad Actors 43\n",
      "Olivia Ruiz Jahiem M 3 Miles 254.74567 free San Antonio-New Braunfels, TX 42 Cabaret Blanco 43\n",
      "Jamie Cullum Jahiem M 0 Miles 204.25098 free San Antonio-New Braunfels, TX 618 Devil May Care 43\n",
      "Renato Carosone Walter M 0 Frye 192.57424 free San Francisco-Oakland-Hayward, CA 1047 Blues (2005 Digital Remaster) 39\n",
      "Lanae' Hale Walter M 0 Frye 195.36934 free San Francisco-Oakland-Hayward, CA 451 Beautiful Things 39\n",
      "blessthefall Lily F 7 Cooper 166.45179 free Columbia, SC 58 Higinia (Album Version) 59\n",
      "Hercules And Love Affair Harper M 37 Barrett 376.29342 paid New York-Newark-Jersey City, NY-NJ-PA 129 Blind (Full Album Version) 42\n",
      "Jonny L Harper M 0 Barrett 401.52771 paid New York-Newark-Jersey City, NY-NJ-PA 275 Microdaze 42\n",
      "Amos Lee Harper M 49 Barrett 178.99057 paid New York-Newark-Jersey City, NY-NJ-PA 404 Baby I Want You 42\n",
      "Rilo Kiley Harper M 12 Barrett 222.06649 paid New York-Newark-Jersey City, NY-NJ-PA 632 With Arms Outstretched (Album Version) 42\n",
      "Sepultura Harper M 10 Barrett 199.23546 paid New York-Newark-Jersey City, NY-NJ-PA 747 Refuse/Resist (Explicit Album Version) 42\n",
      "Kings Of Leon James M 0 Martin 238.8371 free Dallas-Fort Worth-Arlington, TX 761 Happy Alone 79\n",
      "N.E.R.D. James M 0 Martin 242.99057 free Dallas-Fort Worth-Arlington, TX 78 Provider (Remix Radio Edit) 79\n",
      "War Kimber F 1 Norris 244.03546 free New York-Newark-Jersey City, NY-NJ-PA 142 The World Is A Ghetto 47\n",
      "Miranda Cosgrove featuring Drake Bell Kimber F 1 Norris 161.04444 free New York-Newark-Jersey City, NY-NJ-PA 488 Leave It All To Me (Theme from iCarly) 47\n",
      "Bap Zachary M 6 Thomas 277.21098 free New York-Newark-Jersey City, NY-NJ-PA 592 FÃÂ¼r Maria (Album Version) 19\n",
      "Eddie Vedder Zachary M 1 Thomas 236.30322 free New York-Newark-Jersey City, NY-NJ-PA 819 Society 19\n",
      "Coldplay Ava F 1 Robinson 311.27465 free London, KY 108 The Scientist 13\n",
      "George Michael Ava F 0 Robinson 302.602 free London, KY 422 Careless Whisper 13\n",
      "Tub Ring Ava F 1 Robinson 233.69098 free London, KY 908 Invalid 13\n",
      "Pavement Marina F 0 Sutton 99.16036 free Salinas, CA 188 Mercy:The Laundromat 48\n",
      "Jacky Terrasson Marina F 0 Sutton 342.7522 free Salinas, CA 373 Le Jardin d'Hiver 48\n",
      "Dance Gavin Dance Marina F 0 Sutton 218.46159 free Salinas, CA 47 Uneasy Hearts Weigh The Most 48\n",
      "My Chemical Romance Dustin M 7 Lee 311.11791 free Myrtle Beach-Conway-North Myrtle Beach, SC-NC 86 Welcome To The Black Parade (Album Version) 87\n",
      "The Dead Weather Chloe F 9 Cuevas 216.92036 paid San Francisco-Oakland-Hayward, CA 1023 Hang You From The Heavens 49\n",
      "Danity Kane Chloe F 95 Cuevas 247.74485 paid San Francisco-Oakland-Hayward, CA 1041 Damaged [Feat. Fabolous] 49\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "As I Lay Dying Chloe F 45 Cuevas 348.1073 paid San Francisco-Oakland-Hayward, CA 1079 Illusions 49\n",
      "Alliance Ethnik Chloe F 46 Cuevas 195.94404 paid San Francisco-Oakland-Hayward, CA 1096 SinceritÃÂ© Et Jalousie 49\n",
      "Norther Chloe F 22 Cuevas 246.54322 paid San Francisco-Oakland-Hayward, CA 1114 Frozen Angel 49\n",
      "Air Chloe F 1 Cuevas 212.21832 free San Francisco-Oakland-Hayward, CA 179 Playground Love 49\n",
      "Muse Chloe F 3 Cuevas 210.46812 free San Francisco-Oakland-Hayward, CA 195 Pink Ego Box 49\n",
      "DeGarmo & Key Chloe F 1 Cuevas 236.48608 free San Francisco-Oakland-Hayward, CA 224 I'm Accepted (The Pledge Album Version) 49\n",
      "mewithoutYou Chloe F 4 Cuevas 311.562 free San Francisco-Oakland-Hayward, CA 267 Bullet To Binary (Pt. Two) 49\n",
      "Cabaret Voltaire Chloe F 1 Cuevas 430.602 free San Francisco-Oakland-Hayward, CA 274 Playing For Time 49\n",
      "Carlos Y JosÃÂ© Chloe F 0 Cuevas 166.45179 free San Francisco-Oakland-Hayward, CA 336 Soy AlbaÃÂ±il 49\n",
      "Ladytron Chloe F 4 Cuevas 325.14567 free San Francisco-Oakland-Hayward, CA 405 Sugar (Jagz Kooner Mix) 49\n",
      "Sex Slaves Chloe F 12 Cuevas 175.51628 free San Francisco-Oakland-Hayward, CA 437 We're Going Out Tonight 49\n",
      "Rise Against Chloe F 1 Cuevas 169.482 free San Francisco-Oakland-Hayward, CA 469 To Them These Streets Belong 49\n",
      "Kings Of Leon Chloe F 1 Cuevas 201.79546 free San Francisco-Oakland-Hayward, CA 490 Revelry 49\n",
      "Buckcherry Chloe F 3 Cuevas 226.14159 free San Francisco-Oakland-Hayward, CA 496 Sorry (Album Version) 49\n",
      "Quad City DJ's Chloe F 0 Cuevas 451.44771 free San Francisco-Oakland-Hayward, CA 506 C'mon N' Ride It (The Train) (LP Version) 49\n",
      "George Baker Selection Chloe F 6 Cuevas 200.4371 free San Francisco-Oakland-Hayward, CA 528 Morning Sky 49\n",
      "Scandinavian Music Group Chloe F 0 Cuevas 544.88771 free San Francisco-Oakland-Hayward, CA 542 Kun Puut Tekee SeittiÃÂ¤ 49\n",
      "Alliance Ethnik Chloe F 1 Cuevas 265.76934 free San Francisco-Oakland-Hayward, CA 553 Creil City 49\n",
      "Bow Wow & Omarion Chloe F 15 Cuevas 284.83873 paid San Francisco-Oakland-Hayward, CA 568 Girlfriend 49\n",
      "Linkin Park Chloe F 15 Cuevas 189.28281 paid San Francisco-Oakland-Hayward, CA 576 Given Up (Album Version) 49\n",
      "Regina Spektor Chloe F 8 Cuevas 191.08526 paid San Francisco-Oakland-Hayward, CA 606 The Calculation (Album Version) 49\n",
      "The Cinematic Orchestra Chloe F 11 Cuevas 613.74649 paid San Francisco-Oakland-Hayward, CA 621 Burn Out 49\n",
      "Liah Chloe F 35 Cuevas 195.65669 paid San Francisco-Oakland-Hayward, CA 630 Tudo Que Eu Sempre Quis 49\n",
      "Harmonia Chloe F 25 Cuevas 655.77751 paid San Francisco-Oakland-Hayward, CA 636 Sehr kosmisch 49\n",
      "Los Lobos Chloe F 74 Cuevas 247.74485 paid San Francisco-Oakland-Hayward, CA 648 Good Morning Aztlan 49\n",
      "The Police Chloe F 5 Cuevas 289.85424 paid San Francisco-Oakland-Hayward, CA 680 So Lonely 49\n",
      "Gary Stewart Chloe F 8 Cuevas 182.67383 paid San Francisco-Oakland-Hayward, CA 695 Back Sliders Wine 49\n",
      "Freefall Feat. Jan Johnston Chloe F 27 Cuevas 532.11383 paid San Francisco-Oakland-Hayward, CA 708 Skydive (I Feel Wonderful) 49\n",
      "Dropkick Murphys Chloe F 5 Cuevas 292.25751 paid San Francisco-Oakland-Hayward, CA 724 Johnny_ I Hardly Knew Ya 49\n",
      "Florence + The Machine Chloe F 62 Cuevas 168.64608 paid San Francisco-Oakland-Hayward, CA 758 You've Got The Love 49\n",
      "Muse Chloe F 5 Cuevas 209.50159 paid San Francisco-Oakland-Hayward, CA 805 Supermassive Black Hole (Twilight Soundtrack Version) 49\n",
      "Radiohead Chloe F 51 Cuevas 267.20608 paid San Francisco-Oakland-Hayward, CA 816 Subterranean Homesick Alien 49\n",
      "Jack's Mannequin Chloe F 54 Cuevas 223.21587 paid San Francisco-Oakland-Hayward, CA 849 American Love (Album Version) 49\n",
      "Julio Iglesias duet with Willie Nelson Chloe F 20 Cuevas 212.16608 paid San Francisco-Oakland-Hayward, CA 916 To All The Girls I've Loved Before (With Julio Iglesias) 49\n",
      "CafÃÂ© Quijano Chloe F 5 Cuevas 197.32853 paid San Francisco-Oakland-Hayward, CA 923 La Lola 49\n",
      "Chiodos Chloe F 14 Cuevas 226.16771 paid San Francisco-Oakland-Hayward, CA 930 Life Is A Perception Of Your Own Reality (Album Version) 49\n",
      "Blindside Chloe F 8 Cuevas 192.20853 paid San Francisco-Oakland-Hayward, CA 959 Pitiful (LP Version) 49\n",
      "Cameo Chloe F 2 Cuevas 211.98322 paid San Francisco-Oakland-Hayward, CA 960 Word Up! 49\n",
      "Hannah Montana Chloe F 10 Cuevas 174.00118 paid San Francisco-Oakland-Hayward, CA 982 The Best Of Both Worlds 49\n",
      "Black Eyed Peas Chloe F 18 Cuevas 229.61587 paid San Francisco-Oakland-Hayward, CA 987 Let's Get It Started 49\n",
      "Ennio Morricone Joseph M 11 Gutierrez 215.01342 free Columbia, SC 167 Chi Mai (El Profesional) 75\n",
      "Plies Joseph M 0 Gutierrez 278.22975 free Columbia, SC 208 Somebody (Loves You) (Explicit Album Version) 75\n",
      "Matmos Joseph M 0 Gutierrez 1449.11628 free Columbia, SC 284 Supreme Balloon 75\n",
      "The All-American Rejects Joseph M 1 Gutierrez 260.23138 free Columbia, SC 467 Happy Endings 75\n",
      "Escape The Fate Joseph M 3 Gutierrez 259.39546 free Columbia, SC 721 Harder Than You Know 75\n",
      "The All-American Rejects Jordyn F 2 Powell 183.61424 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 357 One More Sad Song 98\n",
      "School Of Seven Bells Jordyn F 1 Powell 176.97914 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 865 s.Ada.Licht 98\n",
      "Kreator Devin M 0 Larson 287.9473 free Tampa-St. Petersburg-Clearwater, FL 271 Second awakening 60\n",
      "Julian Lennon Devin M 2 Larson 210.46812 free Tampa-St. Petersburg-Clearwater, FL 413 Too Late For Goodbyes 60\n",
      "Creedence Clearwater Revival Devin M 2 Larson 271.882 free Tampa-St. Petersburg-Clearwater, FL 531 I Put A Spell On You 60\n",
      "No Doubt Devin M 0 Larson 269.73995 free Tampa-St. Petersburg-Clearwater, FL 59 Sometimes 60\n",
      "Franz Ferdinand Devin M 5 Larson 183.35302 free Tampa-St. Petersburg-Clearwater, FL 624 Fade Together 60\n",
      "Daft Punk Devin M 2 Larson 207.59465 free Tampa-St. Petersburg-Clearwater, FL 746 Aerodynamic 60\n",
      "Aaliyah Devin M 1 Larson 266.50077 free Tampa-St. Petersburg-Clearwater, FL 765 Are You That Somebody? ( LP Version ) 60\n",
      "Fall Out Boy Devin M 1 Larson 180.40118 free Tampa-St. Petersburg-Clearwater, FL 875 Dance_ Dance 60\n",
      "Feist Evelin F 0 Ayala 184.94649 free Milwaukee-Waukesha-West Allis, WI 318 My Moon My Man 34\n",
      "Eminem Evelin F 6 Ayala 284.18567 free Milwaukee-Waukesha-West Allis, WI 33 The Real Slim Shady 34\n",
      "Lupe Fiasco Evelin F 3 Ayala 273.94567 free Milwaukee-Waukesha-West Allis, WI 495 Shining Down [feat. Matthew Santos] (Amended Album Version) 34\n",
      "Dexter Freebish Adelyn F 3 Jordan 210.54649 free Chicago-Naperville-Elgin, IL-IN-WI 391 Deeper 7\n",
      "Explosions In The Sky Adelyn F 0 Jordan 497.47546 free Chicago-Naperville-Elgin, IL-IN-WI 458 Your Hand In Mine 7\n",
      "Static-X Adelyn F 0 Jordan 183.69261 free Chicago-Naperville-Elgin, IL-IN-WI 6 Dirthouse (Album Version) 7\n",
      "Kanye West Makinley F 0 Jones 278.88281 free Chicago-Naperville-Elgin, IL-IN-WI 118 Family Business 17\n",
      "Aaron Shust Makinley F 2 Jones 222.11873 free Chicago-Naperville-Elgin, IL-IN-WI 190 Like I Never Felt Before 17\n",
      "The Ruts Makinley F 5 Jones 338.96444 free Chicago-Naperville-Elgin, IL-IN-WI 732 West One (Shine On Me) 17\n",
      "The Notorious B.I.G. Makinley F 1 Jones 223.73832 free Chicago-Naperville-Elgin, IL-IN-WI 881 Unbelievable (Amended Version) 17\n",
      "Professor Longhair Ann F 0 Banks 214.20363 free Salt Lake City, UT 124 Mean Ol'World 99\n",
      "Astral Projection Ann F 1 Banks 585.42975 free Salt Lake City, UT 227 Flying Into a Star 99\n",
      "Raphael Saadiq featuring Stevie Wonder and CJ Ann F 1 Banks 252.94322 free Salt Lake City, UT 421 Never Give You Up 99\n",
      "59 Times the Pain Jordan F 0 Rodriguez 144.95302 free Los Angeles-Long Beach-Anaheim, CA 136 Found Home 68\n",
      "Copeland Jordan F 0 Rodriguez 279.32689 free Los Angeles-Long Beach-Anaheim, CA 487 The Day I Lost My Voice (The Suitcase Song) 68\n",
      "Shaggy / Ricardo Ducent Jordan F 0 Rodriguez 226.76853 free Los Angeles-Long Beach-Anaheim, CA 843 It Wasn't Me 68\n",
      "Light Of The World Kynnedi F 0 Sanchez 207.0722 free Cedar Rapids, IA 1029 Midnight Groovin' (7\" Version) (2006 Digital Remaster) 89\n",
      "In Flames Kynnedi F 1 Sanchez 255.76444 free Cedar Rapids, IA 192 Come Clarity 89\n",
      "Tony Bennett & k.d. lang Kynnedi F 0 Sanchez 188.26404 free Cedar Rapids, IA 334 That's My Home 89\n",
      "Cake Kynnedi F 1 Sanchez 179.09506 free Cedar Rapids, IA 600 Commissioning A Symphony In C 89\n",
      "Pavement Kynnedi F 0 Sanchez 99.16036 free Cedar Rapids, IA 631 Mercy:The Laundromat 89\n",
      "Cat Stevens Kynnedi F 1 Sanchez 200.202 free Cedar Rapids, IA 88 Wild World 89\n",
      "Alliance Ethnik Kynnedi F 1 Sanchez 195.94404 free Cedar Rapids, IA 927 SinceritÃÂ© Et Jalousie 89\n",
      "Fightstar Wyatt M 1 Scott 212.61016 free Eureka-Arcata-Fortuna, CA 1010 The English Way 9\n",
      "Red Hot Chili Peppers Wyatt M 0 Scott 265.50812 free Eureka-Arcata-Fortuna, CA 335 Under The Bridge (Album Version) 9\n",
      "Nik & Jay Wyatt M 0 Scott 196.51873 free Eureka-Arcata-Fortuna, CA 379 Pop-Pop! 9\n",
      "Train Wyatt M 2 Scott 216.76363 free Eureka-Arcata-Fortuna, CA 527 Hey_ Soul Sister 9\n",
      "Madcon Wyatt M 1 Scott 215.92771 free Eureka-Arcata-Fortuna, CA 551 Beggin' 9\n",
      "Juan Carlos Baglietto Wyatt M 4 Scott 285.64853 free Eureka-Arcata-Fortuna, CA 8 Era En Abril 9\n",
      "10_000 Maniacs Wyatt M 2 Scott 251.8722 free Eureka-Arcata-Fortuna, CA 856 Gun Shy (LP Version) 9\n",
      "AFI Wyatt M 6 Scott 256.86159 free Eureka-Arcata-Fortuna, CA 922 The Interview 9\n",
      "Sneaker Pimps Jayden M 1 Bell 260.91057 free Dallas-Fort Worth-Arlington, TX 1085 Spin Spin Sugar 91\n",
      "Lily Allen Jayden M 3 Bell 240.19546 free Dallas-Fort Worth-Arlington, TX 645 Littlest Things (Live At The Astoria) 91\n",
      "Stephen Lynch Jayden M 0 Bell 182.85669 free Dallas-Fort Worth-Arlington, TX 829 Jim Henson's Dead 91\n",
      "Bon Jovi Jayden M 3 Bell 224.522 free Dallas-Fort Worth-Arlington, TX 90 It's My Life 91\n",
      "Throw Me The Statue Celeste F 3 Williams 189.85751 free Klamath Falls, OR 215 Noises 53\n",
      "Shakira Celeste F 0 Williams 571.16689 free Klamath Falls, OR 311 Estoy AquÃÂ­ 53\n",
      "Charttraxx Karaoke Celeste F 3 Williams 225.17506 free Klamath Falls, OR 438 Fireflies 53\n",
      "Blockhead Celeste F 1 Williams 289.56689 free Klamath Falls, OR 485 The Music Scene 53\n",
      "Jimmy Eat World Celeste F 5 Williams 285.83138 free Klamath Falls, OR 52 Dizzy 53\n",
      "Eminem Celeste F 0 Williams 248.58077 free Klamath Falls, OR 626 Just Lose It 53\n",
      "Bootsy Collins Celeste F 0 Williams 411.42812 free Klamath Falls, OR 826 Stretchin' Out (In A Rubber Band) (LP Version) 53\n",
      "Stacie Orrico Celeste F 1 Williams 225.35791 free Klamath Falls, OR 860 Stuck (Album Version) 53\n",
      "Steppenwolf Celeste F 1 Williams 270.28853 free Klamath Falls, OR 883 Magic Carpet Ride 53\n",
      "Bonobo Lily F 75 Koch 270.41914 paid Chicago-Naperville-Elgin, IL-IN-WI 172 1009 15\n",
      "PeterLicht Lily F 26 Koch 306.80771 paid Chicago-Naperville-Elgin, IL-IN-WI 199 Heiterkeit 15\n",
      "The Cat Empire Lily F 104 Koch 307.46077 paid Chicago-Naperville-Elgin, IL-IN-WI 221 The Chariot 15\n",
      "Frankie HI-NRG MC Lily F 12 Koch 104.09751 paid Chicago-Naperville-Elgin, IL-IN-WI 362 Pace E Guerra 15\n",
      "Drake / Lil Wayne / Young Jeezy Lily F 1 Koch 225.2273 paid Chicago-Naperville-Elgin, IL-IN-WI 417 I'm Goin In 15\n",
      "Dave Grusin Lily F 19 Koch 255.50322 paid Chicago-Naperville-Elgin, IL-IN-WI 557 St. Elsewhere 15\n",
      "Deadmau5 & Kaskade Lily F 4 Koch 595.56526 paid Chicago-Naperville-Elgin, IL-IN-WI 582 I Remember 15\n",
      "Avenged Sevenfold Lily F 9 Koch 426.13506 paid Chicago-Naperville-Elgin, IL-IN-WI 612 Shattered By Broken Dreams 15\n",
      "Plies Lily F 61 Koch 231.6273 paid Chicago-Naperville-Elgin, IL-IN-WI 716 Goons Lurkin (Explicit Album Version) 15\n",
      "Erin Bode Lily F 76 Koch 252.05506 paid Chicago-Naperville-Elgin, IL-IN-WI 764 Here_ There And Everywhere 15\n",
      "Jackson Browne Lily F 37 Koch 367.35955 paid Chicago-Naperville-Elgin, IL-IN-WI 818 Sky Blue And Black (LP Version) 15\n",
      "Mondo Marcio Lily F 23 Koch 263.60118 paid Chicago-Naperville-Elgin, IL-IN-WI 834 Purple Weed 15\n",
      "I Set My Friends On Fire Sylvie F 0 Cruz 206.39302 free Washington-Arlington-Alexandria, DC-VA-MD-WV 1005 But The NUNS Are Watching 10\n",
      "Lonnie Gordon Sylvie F 3 Cruz 181.21098 free Washington-Arlington-Alexandria, DC-VA-MD-WV 182 Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 10\n",
      "Klaus Badelt Sylvie F 1 Cruz 128.62649 free Washington-Arlington-Alexandria, DC-VA-MD-WV 210 Moonlight Serenade 10\n",
      "Nightwish Sylvie F 1 Cruz 305.97179 free Washington-Arlington-Alexandria, DC-VA-MD-WV 220 The Islander 10\n",
      "Amy Winehouse Sylvie F 1 Cruz 202.73587 free Washington-Arlington-Alexandria, DC-VA-MD-WV 272 You Know I'm No Good 10\n",
      "The White Stripes Sylvie F 3 Cruz 231.81016 free Washington-Arlington-Alexandria, DC-VA-MD-WV 286 Seven Nation Army (Album Version) 10\n",
      "Wu-Tang Clan Sylvie F 2 Cruz 122.46159 free Washington-Arlington-Alexandria, DC-VA-MD-WV 339 Impossible 10\n",
      "Pavement Sylvie F 0 Cruz 99.16036 free Washington-Arlington-Alexandria, DC-VA-MD-WV 345 Mercy:The Laundromat 10\n",
      "Ricardo Arjona Sylvie F 7 Cruz 210.25914 free Washington-Arlington-Alexandria, DC-VA-MD-WV 484 Como Duele (Album) 10\n",
      "Cameo Sylvie F 0 Cruz 211.98322 free Washington-Arlington-Alexandria, DC-VA-MD-WV 577 Word Up! 10\n",
      "Samael Sylvie F 0 Cruz 247.09179 free Washington-Arlington-Alexandria, DC-VA-MD-WV 677 God's Snake 10\n",
      "Brad Paisley With Andy Griffith Sylvie F 0 Cruz 302.70649 free Washington-Arlington-Alexandria, DC-VA-MD-WV 681 Waitin' On A Woman 10\n",
      "Chromeo Sylvie F 0 Cruz 171.57179 free Washington-Arlington-Alexandria, DC-VA-MD-WV 717 Momma's Boy 10\n",
      "Matt Redman Sylvie F 0 Cruz 303.46404 free Washington-Arlington-Alexandria, DC-VA-MD-WV 864 Blessed Be Your Name 10\n",
      "John Mayer Sylvie F 1 Cruz 201.56036 free Washington-Arlington-Alexandria, DC-VA-MD-WV 877 Love Song For No One 10\n",
      "Black Eyed Peas Sylvie F 0 Cruz 214.93506 free Washington-Arlington-Alexandria, DC-VA-MD-WV 9 Pump It 10\n",
      "USS (Ubiquitous Synergy Seeker) Amiya F 22 Davidson 307.61751 paid Longview, TX 64 Man Makes The Zoo 65\n",
      "Metallica Ryann F 1 Smith 466.54649 free Palestine, TX 1006 The Unforgiven III 92\n",
      "Bajaga & Instruktori Ryann F 3 Smith 254.95465 free Palestine, TX 1092 Godine prolaze 92\n",
      "Red Hot Chili Peppers Ryann F 4 Smith 231.33995 free Palestine, TX 328 Easily (Album Version) 92\n",
      "Hot Chip Ryann F 2 Smith 280.5024 free Palestine, TX 468 Don't Dance 92\n",
      "Ratatat Ryann F 0 Smith 184.0322 free Palestine, TX 825 Nostrand 92\n",
      "Vampire Weekend Ryann F 2 Smith 255.81669 free Palestine, TX 905 M79 (Album) 92\n",
      "Usher featuring will.i.am Ryann F 2 Smith 395.72853 free Palestine, TX 91 OMG 92\n",
      "Pixies Ryann F 6 Smith 210.99057 free Palestine, TX 938 Hey 92\n",
      "Arctic Monkeys Ryann F 3 Smith 175.43791 free Palestine, TX 964 Mardy Bum 92\n",
      "King Cobb Steelie Theodore M 0 Smith 373.7073 free Houston-The Woodlands-Sugar Land, TX 1055 Starvo 52\n",
      "Foo Fighters Theodore M 1 Smith 262.73914 free Houston-The Woodlands-Sugar Land, TX 226 All My Life 52\n",
      "Eddie Cochran Theodore M 1 Smith 113.68444 free Houston-The Woodlands-Sugar Land, TX 261 Mean When I'm Mad 52\n",
      "Camila Theodore M 0 Smith 252.29016 free Houston-The Woodlands-Sugar Land, TX 299 Besame 52\n",
      "Earth_ Wind & Fire Theodore M 1 Smith 178.20689 free Houston-The Woodlands-Sugar Land, TX 447 Night Dreamin' 52\n",
      "Dave Hollister Theodore M 1 Smith 271.62077 free Houston-The Woodlands-Sugar Land, TX 601 One Woman Man 52\n",
      "Florence + The Machine Theodore M 0 Smith 131.3171 free Houston-The Woodlands-Sugar Land, TX 796 Kiss With A Fist 52\n",
      "Bond Theodore M 1 Smith 208.16934 free Houston-The Woodlands-Sugar Land, TX 904 Victory 52\n",
      "Kostia Theodore M 2 Smith 281.10322 free Houston-The Woodlands-Sugar Land, TX 953 Secret Garden 52\n",
      "Spoon Theodore M 0 Smith 299.20608 free Houston-The Woodlands-Sugar Land, TX 968 The Mystery Zone 52\n",
      "Justin Timberlake Theodore M 2 Smith 313.0771 free Houston-The Woodlands-Sugar Land, TX 990 What Goes Around...Comes Around 52\n",
      "James Newton Howard Alivia F 0 Terrell 141.5571 free Parkersburg-Vienna, WV 1054 I'm Sorry 4\n",
      "The Killers Alivia F 1 Terrell 246.80444 free Parkersburg-Vienna, WV 3 Read My Mind 4\n",
      "The Spill Canvas Alivia F 1 Terrell 204.90404 free Parkersburg-Vienna, WV 572 Bracelets (LP Version) 4\n",
      "Cedell Davis Alivia F 0 Terrell 178.36363 free Parkersburg-Vienna, WV 587 Chicken Hawk 4\n",
      "A Perfect Circle Kinsley F 20 Young 291.52608 paid Red Bluff, CA 368 Vanishing 85\n",
      "Deborah Harry Kinsley F 3 Young 257.41016 paid Red Bluff, CA 427 Kiss It Better 85\n",
      "Scissor Sisters Kinsley F 37 Young 156.94322 paid Red Bluff, CA 436 Intermission 85\n",
      "Taylor Swift Kinsley F 24 Young 246.282 paid Red Bluff, CA 658 The Best Day 85\n",
      "Hans Zimmer_ James Newton Howard Kinsley F 8 Young 265.50812 paid Red Bluff, CA 759 Nycteris 85\n",
      "Denise Jannah Kinsley F 35 Young 477.04771 paid Red Bluff, CA 776 'Round Midnight 85\n",
      "The Kills Kinsley F 5 Young 231.91465 paid Red Bluff, CA 793 Goodnight Bad Morning 85\n",
      "Nicolai Dunger Kinsley F 29 Young 163.65669 paid Red Bluff, CA 891 Our Filmscore 85\n",
      "Sugababes Kinsley F 33 Young 275.93098 paid Red Bluff, CA 977 Overload 85\n",
      "Parachute Dominick M 0 Norris 146.05016 free Los Angeles-Long Beach-Anaheim, CA 403 She Is Love 45\n",
      "Beirut Cienna F 1 Freeman 142.31465 free New York-Newark-Jersey City, NY-NJ-PA 131 The Penalty 56\n",
      "The Rakes Isaac M 2 Valdez 150.59546 free Saginaw, MI 112 Strasbourg 3\n",
      "La Polla Records Anabelle F 4 Simpson 190.77179 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 1019 Eutanasia 69\n",
      "Maldita Nerea Anabelle F 0 Simpson 241.162 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 158 Supelicula 69\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Aesop Rock Anabelle F 1 Simpson 245.65506 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 235 None Shall Pass (Main) 69\n",
      "The Killers Anabelle F 2 Simpson 245.99465 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 249 Sam's Town 69\n",
      "A Fine Frenzy Anabelle F 0 Simpson 267.91138 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 256 Almost Lover (Album Version) 69\n",
      "Alicia Keys Anabelle F 1 Simpson 229.0673 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 259 If I Ain't Got You 69\n",
      "California Swag District Anabelle F 3 Simpson 239.17669 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 264 Teach Me How To Dougie 69\n",
      "Laura Izibor Anabelle F 0 Simpson 211.56526 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 378 Carousel (PSILY Album Version) 69\n",
      "Biffy Clyro Anabelle F 2 Simpson 189.83138 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 455 God & Satan 69\n",
      "Daft Punk Anabelle F 0 Simpson 396.82567 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 465 Da Funk / Dadftendirekt 69\n",
      "Iron Maiden Anabelle F 0 Simpson 227.76118 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 651 Holy Smoke 69\n",
      "Ivy Anabelle F 1 Simpson 264.61995 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 791 Edge Of The Ocean 69\n",
      "Yann Tiersen Anabelle F 0 Simpson 120.39791 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 858 La Valse D'AmÃÂ©lie (Version Orchestre) 69\n",
      "Alliance Ethnik Anabelle F 2 Simpson 252.21179 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 890 ReprÃÂ©sente 69\n",
      "R. Kelly Anabelle F 1 Simpson 234.39628 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 901 The World's Greatest 69\n",
      "Florence + The Machine Anabelle F 1 Simpson 176.69179 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 926 My Boy Builds Coffins 69\n",
      "Muse Avery F 15 Watkins 304.84853 paid San Jose-Sunnyvale-Santa Clara, CA 130 Uprising 30\n",
      "Descendents Avery F 111 Watkins 89.23383 paid San Jose-Sunnyvale-Santa Clara, CA 324 I'm Not A Loser 30\n",
      "Red Hot Chili Peppers Avery F 9 Watkins 334.65424 paid San Jose-Sunnyvale-Santa Clara, CA 629 Snow [Hey Oh] (Album Version) 30\n",
      "Prince & The Revolution Avery F 32 Watkins 227.65669 paid San Jose-Sunnyvale-Santa Clara, CA 691 Kiss (LP Version) 30\n",
      "Lagwagon Avery F 20 Watkins 173.322 paid San Jose-Sunnyvale-Santa Clara, CA 884 Sick 30\n",
      "Pat Metheny Group Avery F 50 Martinez 324.8322 paid Atlanta-Sandy Springs-Roswell, GA 1017 Always and Forever 82\n",
      "Enrique Iglesias Avery F 54 Martinez 217.99138 paid Atlanta-Sandy Springs-Roswell, GA 140 DÃÂ­melo 82\n",
      "California Swag District Hayden F 38 Brock 239.17669 paid Detroit-Warren-Dearborn, MI 117 Teach Me How To Dougie 72\n",
      "Timbaland / Mia Hayden F 35 Brock 234.52689 paid Detroit-Warren-Dearborn, MI 381 Come Around 72\n",
      "Andrew Bird Elijah M 2 Davis 316.47302 free Detroit-Warren-Dearborn, MI 138 Skin Is_ My 5\n",
      "Deas Vail Elijah M 0 Davis 237.68771 free Detroit-Warren-Dearborn, MI 985 Anything You Say (Unreleased Version) 5\n",
      "The Mercury Program Adler M 2 Barrera 449.12281 free New York-Newark-Jersey City, NY-NJ-PA 174 Marianas 100\n",
      "Klaus Doldinger's Passport Adler M 3 Barrera 460.43383 free New York-Newark-Jersey City, NY-NJ-PA 198 Fairy Tale 100\n",
      "Limp Bizkit Adler M 2 Barrera 270.49751 free New York-Newark-Jersey City, NY-NJ-PA 248 Behind Blue Eyes 100\n",
      "Jagged Edge featuring Run of Run DMC Adler M 1 Barrera 248.37179 free New York-Newark-Jersey City, NY-NJ-PA 301 Let's Get Married 100\n",
      "Linkin Park Adler M 0 Barrera 164.41424 free New York-Newark-Jersey City, NY-NJ-PA 341 Bleed It Out (Album Version) 100\n",
      "Vanessa Carlton Adler M 4 Barrera 272.40444 free New York-Newark-Jersey City, NY-NJ-PA 428 Come Undone 100\n",
      "Linkin Park Adler M 1 Barrera 268.61669 free New York-Newark-Jersey City, NY-NJ-PA 493 New Divide (Album Version) 100\n",
      "Lonnie Gordon Adler M 1 Barrera 181.21098 free New York-Newark-Jersey City, NY-NJ-PA 595 Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 100\n",
      "Concrete Blonde Adler M 0 Barrera 270.28853 free New York-Newark-Jersey City, NY-NJ-PA 942 Bloodletting (The Vampire Song) 100\n",
      "Milli Vanilli Katherine F 0 Gay 197.92934 free San Antonio-New Braunfels, TX 1020 Girl You Know It's True 57\n",
      "Coldplay Katherine F 1 Gay 268.38159 free San Antonio-New Braunfels, TX 266 Yellow 57\n",
      "Michael Franks Katherine F 0 Gay 282.01751 free San Antonio-New Braunfels, TX 321 Rainy Night In Tokyo (Remastered LP Version) 57\n",
      "Arctic Monkeys Katherine F 3 Gay 177.94567 free San Antonio-New Braunfels, TX 56 Fake Tales Of San Francisco 57\n",
      "The All-American Rejects Jayden F 4 Duffy 238.86322 free Seattle-Tacoma-Bellevue, WA 509 Move Along 76\n",
      "Nancy Sinatra Jayden F 2 Duffy 184.63302 free Seattle-Tacoma-Bellevue, WA 561 So Long_ Babe (2006 Digital Remaster) 76\n",
      "Fischer-Z Jayden F 3 Duffy 302.78485 free Seattle-Tacoma-Bellevue, WA 640 So Long 76\n",
      "Anna Waronker Jayden F 0 Duffy 189.6224 free Seattle-Tacoma-Bellevue, WA 662 Nothing Personal 76\n",
      "Fatboy Slim Jayden F 2 Duffy 192.9922 free Seattle-Tacoma-Bellevue, WA 75 Illuminati 76\n",
      "The Lonely Island / T-Pain Jayden F 1 Duffy 156.23791 free Seattle-Tacoma-Bellevue, WA 846 I'm On A Boat 76\n",
      "Die Mooskirchner Jayden F 1 Duffy 169.29914 free Seattle-Tacoma-Bellevue, WA 921 Frisch und g'sund 76\n",
      "Amy Winehouse Matthew M 3 Jones 165.11955 paid Janesville-Beloit, WI 1100 Addicted 36\n",
      "Fat Joe Matthew M 7 Jones 317.09995 free Janesville-Beloit, WI 178 Story To Tell 36\n",
      "Pulp Matthew M 1 Jones 244.08771 free Janesville-Beloit, WI 388 Babies 36\n",
      "Mayday Parade Matthew M 65 Jones 316.9171 paid Janesville-Beloit, WI 392 Miserable At Best (Album) 36\n",
      "Ron Carter Matthew M 19 Jones 497.13587 paid Janesville-Beloit, WI 439 I CAN'T GET STARTED 36\n",
      "Interpol Matthew M 55 Jones 299.72853 paid Janesville-Beloit, WI 461 Rest My Chemistry 36\n",
      "Dilated Peoples Featuring Kanye West Matthew M 18 Jones 245.15873 paid Janesville-Beloit, WI 728 This Way 36\n",
      "Charlotte Sometimes Matthew M 7 Jones 173.68771 paid Janesville-Beloit, WI 808 How I Could Just Kill A Man 36\n",
      "D Black Matthew M 76 Jones 261.14567 paid Janesville-Beloit, WI 957 Mais E Mais Amor 36\n",
      "Busted / Charlie Simpson / Mattie Jay / James Bourne Matthew M 24 Jones 252.83873 paid Janesville-Beloit, WI 998 Can't Break Thru 36\n",
      "Yellowcard Samuel M 1 Gonzalez 208.79628 free Houston-The Woodlands-Sugar Land, TX 1064 One Year_ Six Months 61\n",
      "Mikel Erentxun Samuel M 1 Gonzalez 178.83383 free Houston-The Woodlands-Sugar Land, TX 252 Frases Mudas 61\n",
      "Spike Milligan Samuel M 0 Gonzalez 220.39465 free Houston-The Woodlands-Sugar Land, TX 384 Nothing At All 61\n",
      "Bryan Adams Samuel M 0 Gonzalez 254.79791 free Houston-The Woodlands-Sugar Land, TX 454 Cloud Number Nine 61\n",
      "Sam Cooke Samuel M 3 Gonzalez 122.04363 free Houston-The Woodlands-Sugar Land, TX 497 Ain't Misbehavin 61\n",
      "Jimmy Eat World Samuel M 2 Gonzalez 385.802 free Houston-The Woodlands-Sugar Land, TX 504 Drugs Or Me 61\n",
      "Sony Wonder Samuel M 0 Gonzalez 218.06975 free Houston-The Woodlands-Sugar Land, TX 597 Blackbird 61\n",
      "Kierra Sheard Samuel M 1 Gonzalez 299.02322 free Houston-The Woodlands-Sugar Land, TX 60 Love Like Crazy (Extended Mix) 61\n",
      "Psychic Ills Samuel M 0 Gonzalez 512.96608 free Houston-The Woodlands-Sugar Land, TX 610 I Knew My Name 61\n",
      "The String Cheese Incident Samuel M 1 Gonzalez 392.35873 free Houston-The Woodlands-Sugar Land, TX 690 Bigger Isn't Better 61\n",
      "Paul Simon Samuel M 0 Gonzalez 189.72689 free Houston-The Woodlands-Sugar Land, TX 703 The Sound Of Silence 61\n",
      "Ticanaf Samuel M 1 Gonzalez 2594.87302 free Houston-The Woodlands-Sugar Land, TX 748 The Thousand Names of Lord Shiva (Part 1) 61\n",
      "Florence + The Machine Samuel M 7 Gonzalez 80.87465 free Houston-The Woodlands-Sugar Land, TX 876 Bird Song Intro 61\n",
      "Plastilina Mosh Kate F 48 Harrell 261.45914 paid Lansing-East Lansing, MI 147 Mr. P-Mosh 97\n",
      "Rick Ross Kate F 102 Harrell 277.36771 paid Lansing-East Lansing, MI 293 Luxury Tax 97\n",
      "Terrorvision Kate F 11 Harrell 160.54812 paid Lansing-East Lansing, MI 374 Glad All Over 97\n",
      "The Black Ghosts Kate F 10 Harrell 203.28444 paid Lansing-East Lansing, MI 508 Anyway You Choose To Give It (Radio Edit) 97\n",
      "Static-X Kate F 57 Harrell 266.47465 paid Lansing-East Lansing, MI 537 The Trance Is The Motion [Live] 97\n",
      "Holy Fuck Kate F 11 Harrell 428.72118 paid Lansing-East Lansing, MI 558 Tonebank Computer 97\n",
      "MichÃÂ¨le Arnaud Kate F 3 Harrell 105.50812 paid Lansing-East Lansing, MI 584 Chanson Sur Une Seule Note Samba De Uma Nota So 97\n",
      "Michael BublÃÂ© Kate F 90 Harrell 229.04118 paid Lansing-East Lansing, MI 605 Can't Help Falling In Love (Album Version) (Studio) 97\n",
      "The Perishers Kate F 35 Harrell 234.16118 paid Lansing-East Lansing, MI 633 Pills (Live) 97\n",
      "Coldplay Kate F 16 Harrell 291.60444 paid Lansing-East Lansing, MI 671 Speed Of Sound 97\n",
      "keller williams Kate F 61 Harrell 230.39955 paid Lansing-East Lansing, MI 797 221 97\n",
      "Steinski Kate F 10 Harrell 101.72036 paid Lansing-East Lansing, MI 806 Product Of The Environment (Redfern Gowanus Electro Mix) 97\n",
      "Adina Howard Kate F 48 Harrell 236.40771 paid Lansing-East Lansing, MI 817 Horny For Your Love (LP Version) 97\n",
      "Phoenix Kate F 28 Harrell 192.86159 paid Lansing-East Lansing, MI 828 Napoleon Says 97\n",
      "Two Door Cinema Club Kate F 7 Harrell 207.43791 paid Lansing-East Lansing, MI 837 Costume Party 97\n",
      "Big D and The Kids Table Kate F 40 Harrell 184.842 paid Lansing-East Lansing, MI 944 Shining On (feat. Scott Grella) (Grella's Techno Remix) 97\n",
      "Streetlight Manifesto Ava F 0 Robinson 303.33342 free New Haven-Milford, CT 1011 Down_ Down_ Down To Mehphisto's Cafe (Album Version) 50\n",
      "Infant Sorrow Ava F 2 Robinson 170.16118 free New Haven-Milford, CT 1050 Inside Of You 50\n",
      "Tony Rohr Ava F 1 Robinson 403.09506 free New Haven-Milford, CT 156 Van Comp 50\n",
      "Dwight Yoakam Ava F 2 Robinson 239.3073 free New Haven-Milford, CT 207 You're The One 50\n",
      "Sade Ava F 2 Robinson 271.96036 free New Haven-Milford, CT 211 Hang On To Your Love 50\n",
      "Slim Cessna's Auto Club Ava F 3 Robinson 333.37424 free New Haven-Milford, CT 313 Unto The Day 50\n",
      "Faithless Ava F 4 Robinson 495.3073 free New Haven-Milford, CT 338 Music Matters (Mark Knight Dub) 50\n",
      "Sondre Lerche Ava F 1 Robinson 171.59791 free New Haven-Milford, CT 343 Heartbeat Radio 50\n",
      "M83 Ava F 4 Robinson 367.5424 free New Haven-Milford, CT 360 Gone 50\n",
      "Muse Ava F 2 Robinson 346.69669 free New Haven-Milford, CT 430 Resistance 50\n",
      "The Killers Ava F 1 Robinson 225.12281 free New Haven-Milford, CT 538 A Dustland Fairytale 50\n",
      "Arctic Monkeys Ava F 1 Robinson 224.83546 free New Haven-Milford, CT 578 Crying Lightning 50\n",
      "New Found Glory Ava F 2 Robinson 222.69342 free New Haven-Milford, CT 598 the king of wishful thinking 50\n",
      "The Wallflowers Ava F 4 Robinson 312.63302 free New Haven-Milford, CT 623 One Headlight 50\n",
      "Kate Ryan Ava F 0 Robinson 209.60608 free New Haven-Milford, CT 653 La Promesse (Radio Version) 50\n",
      "Eagles Ava F 0 Robinson 181.86404 free New Haven-Milford, CT 667 Earlybird (LP Version) 50\n",
      "Matt Costa Ava F 0 Robinson 246.02077 free New Haven-Milford, CT 669 Unfamiliar Faces 50\n",
      "El Coyote Y Su Banda Tierra Santa Ava F 0 Robinson 190.51057 free New Haven-Milford, CT 694 Me Voy A Ir 50\n",
      "K'Naan Ava F 0 Robinson 220.49914 free New Haven-Milford, CT 824 Wavin'  Flag 50\n",
      "Jill Barber Ava F 0 Robinson 199.52281 free New Haven-Milford, CT 836 Two Brown Eyes 50\n",
      "Traveling Wilburys Ava F 0 Robinson 207.59465 free New Haven-Milford, CT 940 End Of The Line 50\n",
      "The Archies Ava F 1 Robinson 166.16444 free New Haven-Milford, CT 979 Sugar Sugar 50\n",
      "Stone Temple Pilots Ava F 0 Robinson 310.25587 free New Haven-Milford, CT 988 Dead & Bloated (LP Version) 50\n",
      "Edward Sharpe & The Magnetic Zeros Jayden M 40 Graves 306.31138 paid Marinette, WI-MI 128 Home 25\n",
      "Linkin Park Jayden M 23 Graves 289.90649 paid Marinette, WI-MI 231 Shadow Of The Day (Album Version) 25\n",
      "Smokey Robinson & The Miracles Jayden M 13 Graves 168.9073 paid Marinette, WI-MI 297 I Second That Emotion 25\n",
      "Eminem Jayden M 26 Graves 250.8273 paid Marinette, WI-MI 442 Mockingbird 25\n",
      "Hollywood Undead Jayden M 9 Graves 185.20771 paid Marinette, WI-MI 534 No. 5 25\n",
      "Redbone Jayden M 31 Graves 586.21342 paid Marinette, WI-MI 594 Sizzle Gets It Again 25\n",
      "Alicia Keys Molly F 7 Taylor 229.0673 free St. Louis, MO-IL 171 If I Ain't Got You 35\n",
      "Oleta Adams Molly F 0 Taylor 281.93914 free St. Louis, MO-IL 290 My Heart Won't Lie 35\n",
      "Stellar Kart Molly F 1 Taylor 186.17424 free St. Louis, MO-IL 396 Jesus Loves You (Album Version) 35\n",
      "K'Naan Molly F 2 Taylor 220.49914 free St. Louis, MO-IL 464 Wavin'  Flag 35\n",
      "Charisma Molly F 0 Taylor 707.63057 free St. Louis, MO-IL 482 Loon Garden 35\n",
      "Cam'Ron / Juelz Santana / Un Kasa Molly F 3 Taylor 228.38812 free St. Louis, MO-IL 634 Take Em To Church 35\n",
      "Alliance Ethnik Molly F 1 Taylor 252.21179 free St. Louis, MO-IL 702 ReprÃÂ©sente 35\n",
      "Boney James Molly F 0 Taylor 327.3922 free St. Louis, MO-IL 740 Seduction (Album Version) 35\n",
      "Michael B. Jacob M 1 Rogers 197.3024 free San Diego-Carlsbad, CA 17 Geh Deinen Weg 18\n",
      "Linkin Park Jacob M 1 Rogers 494.99383 free San Diego-Carlsbad, CA 425 Bleed It Out [Live At Milton Keynes] 18\n",
      "Jamiroquai Jacob M 0 Rogers 362.05669 free San Diego-Carlsbad, CA 432 Talullah 18\n",
      "Cro-Mags Jacob M 0 Rogers 206.99383 free San Diego-Carlsbad, CA 459 Eyes Of Tomorrow 18\n",
      "Soda Stereo Braden M 0 Parker 316.73424 free Youngstown-Warren-Boardman, OH-PA 1007 Signos 74\n",
      "The All-American Rejects Braden M 0 Parker 206.96771 free Youngstown-Warren-Boardman, OH-PA 1082 Fallin' Apart 74\n",
      "Was (Not Was) Braden M 1 Parker 239.15057 free Youngstown-Warren-Boardman, OH-PA 164 Shake Your Head 74\n",
      "Ryan Adams Braden M 0 Parker 248.5024 free Youngstown-Warren-Boardman, OH-PA 246 Wonderwall 74\n",
      "Crystal Castles Braden M 1 Parker 293.8771 free Youngstown-Warren-Boardman, OH-PA 570 Year Of Silence 74\n",
      "Something Corporate Braden M 1 Parker 255.18975 free Youngstown-Warren-Boardman, OH-PA 749 If You C Jordan 74\n",
      "tobyMac Braden M 0 Parker 229.69424 free Youngstown-Warren-Boardman, OH-PA 801 Burn For You 74\n",
      "Blue States Brantley M 4 West 239.3073 free Portland-Vancouver-Hillsboro, OR-WA 133 Allies 28\n",
      "Kid Cudi Vs Crookers Brantley M 0 West 162.97751 free Portland-Vancouver-Hillsboro, OR-WA 242 Day 'N' Nite 28\n",
      "Linkin Park Brantley M 1 West 208.95302 free Portland-Vancouver-Hillsboro, OR-WA 270 Crawling (Album Version) 28\n",
      "Bill Withers Theodore M 1 Harris 383.73832 free Red Bluff, CA 1038 Make Love To Your Mind 14\n",
      "Oscar Peterson Theodore M 3 Harris 516.70159 free Red Bluff, CA 162 Nighttime 14\n",
      "Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner Theodore M 0 Harris 277.15873 free Red Bluff, CA 340 Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile) 14\n",
      "Mobb Deep Theodore M 1 Harris 181.15873 free Red Bluff, CA 418 Shook Ones pt. 2 14\n",
      "Lifehouse Theodore M 2 Harris 195.47383 free Red Bluff, CA 440 You And Me (Wedding Version) 14\n",
      "The Van Pelt Theodore M 1 Harris 208.71791 free Red Bluff, CA 462 It's New To Me 14\n",
      "Stephane Pompougnac Theodore M 1 Harris 241.10975 free Red Bluff, CA 478 Ghosts and Roses 14\n",
      "Okkervil River Theodore M 2 Harris 223.13751 free Red Bluff, CA 515 Plus Ones 14\n",
      "Florence + The Machine Theodore M 2 Harris 219.66322 free Red Bluff, CA 543 Dog Days Are Over (Radio Edit) 14\n",
      "Burt Bacharach Theodore M 1 Harris 156.96934 free Red Bluff, CA 813 Casino Royale Theme (Main Title) 14\n",
      "Gary Allan Theodore M 2 Harris 244.34893 free Red Bluff, CA 929 Best I Ever Had 14\n",
      "Girl Talk Kaylee F 8 Summers 160.15628 free Phoenix-Mesa-Scottsdale, AZ 139 Once again 8\n",
      "4 Skins Kaylee F 4 Summers 165.51138 free Phoenix-Mesa-Scottsdale, AZ 181 Clockwork Skinhead 8\n",
      "ISRAEL & NEW BREED Kaylee F 1 Summers 176.48281 free Phoenix-Mesa-Scottsdale, AZ 333 Awesome Medley 8\n",
      "Johnny Cash with The Carter Family The Statler Brothers & Carl Perkins Kaylee F 3 Summers 308.34893 free Phoenix-Mesa-Scottsdale, AZ 532 Closing Medley: Folsom Prison Blues/I Walk The Line/Ring Of Fire/The Rebel - Johnny Yuma 8\n",
      "Battles Kaylee F 1 Summers 118.85669 free Phoenix-Mesa-Scottsdale, AZ 661 Snare Hangar 8\n",
      "Cosmic Gate Kaylee F 0 Summers 652.59057 free Phoenix-Mesa-Scottsdale, AZ 675 I Feel Wonderful (AM to PM Mix) 8\n",
      "John Mayer Kaylee F 1 Summers 255.11138 free Phoenix-Mesa-Scottsdale, AZ 705 War Of My Life 8\n",
      "Linkin Park Kaylee F 5 Summers 189.28281 free Phoenix-Mesa-Scottsdale, AZ 833 Given Up (Album Version) 8\n",
      "Alejandro Sanz Maia F 1 Burke 230.24281 free Houston-The Woodlands-Sugar Land, TX 143 Non E_ Per Te_ Per Me 51\n",
      "Enigma Maia F 6 Burke 141.5571 free Houston-The Woodlands-Sugar Land, TX 316 The Voice Of Enigma 51\n",
      "Angus & Julia Stone Maia F 0 Burke 262.42567 free Houston-The Woodlands-Sugar Land, TX 502 Jewels And Gold 51\n",
      "All Time Low Maia F 1 Burke 177.84118 free Houston-The Woodlands-Sugar Land, TX 510 A Party Song (The Walk of Shame) 51\n",
      "La Roux Maia F 2 Burke 208.03873 free Houston-The Woodlands-Sugar Land, TX 526 Colourless Colour 51\n",
      "Dislocation Dance Maia F 0 Burke 122.87955 free Houston-The Woodlands-Sugar Land, TX 628 With A Smile On Your Face And A Frown In Your Heart 51\n",
      "Fluke Connar M 0 Moreno 478.92853 free Houston-The Woodlands-Sugar Land, TX 168 Bermuda 62\n",
      "Plaid Connar M 3 Moreno 260.96281 free Houston-The Woodlands-Sugar Land, TX 218 Eyen [Chosen by fans on Warp20.net] 62\n",
      "Olivia Newton-John Connar M 1 Moreno 132.23138 free Houston-The Woodlands-Sugar Land, TX 476 Hochmah (Interlude) 62\n",
      "Caifanes Connar M 3 Moreno 273.99791 free Houston-The Woodlands-Sugar Land, TX 585 Nubes 62\n",
      "Bitter:Sweet Jaleah F 18 Hayes 160.78322 paid San Antonio-New Braunfels, TX 113 Take 2 Blue 70\n",
      "Sade Jaleah F 14 Hayes 236.40771 paid San Antonio-New Braunfels, TX 696 Nothing Can Come Between Us 70\n",
      "Matthew Good Aleena F 4 Kirby 319.16363 paid Waterloo-Cedar Falls, IA 1003 Born Losers 44\n",
      "Bright Eyes Aleena F 0 Kirby 200.72444 paid Waterloo-Cedar Falls, IA 1069 Take It Easy (love Nothing) (Album Version) 44\n",
      "Covenant Aleena F 15 Kirby 300.19873 paid Waterloo-Cedar Falls, IA 157 XRDS 44\n",
      "Vampire Weekend Aleena F 46 Kirby 195.7873 paid Waterloo-Cedar Falls, IA 196 Oxford Comma (Album) 44\n",
      "Mediaeval Baebes Aleena F 23 Kirby 219.29751 paid Waterloo-Cedar Falls, IA 237 MÃÂ¤rk Hure VÃÂ¥r Skugga 44\n",
      "Bill Withers Aleena F 85 Kirby 383.73832 paid Waterloo-Cedar Falls, IA 269 Make Love To Your Mind 44\n",
      "Avril Lavigne Aleena F 7 Kirby 212.40118 paid Waterloo-Cedar Falls, IA 285 Tomorrow 44\n",
      "Muse Aleena F 35 Kirby 209.34485 paid Waterloo-Cedar Falls, IA 350 Supermassive Black Hole (Album Version) 44\n",
      "Passion Pit Aleena F 13 Kirby 243.69587 paid Waterloo-Cedar Falls, IA 448 Eyes As Candles 44\n",
      "Subhumans Aleena F 16 Kirby 209.37098 paid Waterloo-Cedar Falls, IA 474 Rain 44\n",
      "Ugly Kid Joe Aleena F 81 Kirby 241.03138 paid Waterloo-Cedar Falls, IA 619 Cats In The Cradle 44\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Nirvana Aleena F 18 Kirby 257.01832 paid Waterloo-Cedar Falls, IA 637 Lithium 44\n",
      "Randy Crawford Aleena F 55 Kirby 270.75873 paid Waterloo-Cedar Falls, IA 639 Almaz 44\n",
      "T.I. Aleena F 23 Kirby 185.10322 paid Waterloo-Cedar Falls, IA 781 Look What I Got (Edited Album Version) 44\n",
      "Postal Service Jacob M 17 Klein 307.53914 paid Tampa-St. Petersburg-Clearwater, FL 1049 Natural Anthem (Album) 73\n",
      "The Yardbirds Jacob M 39 Klein 206.10567 paid Tampa-St. Petersburg-Clearwater, FL 255 The Train Kept A Rollin' 73\n",
      "Los AutÃÂ©nticos Decadentes Jacob M 59 Klein 218.48771 paid Tampa-St. Petersburg-Clearwater, FL 294 La Guitarra 73\n",
      "Drake / Kanye West / Lil Wayne / Eminem Jacob M 50 Klein 357.66812 paid Tampa-St. Petersburg-Clearwater, FL 518 Forever 73\n",
      "Dave Brockie Experience Jacob M 49 Klein 116.08771 paid Tampa-St. Petersburg-Clearwater, FL 692 Servant Of Death's Head 73\n",
      "Beatsteaks Jacob M 27 Klein 177.8673 paid Tampa-St. Petersburg-Clearwater, FL 72 Jane Became Insane 73\n",
      "Vangelis Jacob M 0 Klein 327.52281 paid Tampa-St. Petersburg-Clearwater, FL 913 Wait For Me 73\n",
      "Bouncing Souls Jacob M 56 Klein 117.31546 paid Tampa-St. Petersburg-Clearwater, FL 954 The Screamer 73\n",
      "Dalto Aiden M 1 Hess 190.40608 free La Crosse-Onalaska, WI-MN 170 Falta Te Dizer 86\n",
      "You Me At Six Aiden M 0 Hess 191.13751 free La Crosse-Onalaska, WI-MN 189 Liquid Confidence 86\n",
      "Dave Darell Aiden M 0 Hess 217.33832 free La Crosse-Onalaska, WI-MN 197 Children 86\n",
      "Kings Of Leon Aiden M 0 Hess 227.13424 free La Crosse-Onalaska, WI-MN 204 Be Somebody 86\n",
      "The Black Keys Aiden M 4 Hess 139.51955 free La Crosse-Onalaska, WI-MN 344 I'll Be Your Man 86\n",
      "Toto La Momposina Aiden M 2 Hess 299.02322 free La Crosse-Onalaska, WI-MN 498 Curura 86\n",
      "Pedro Aznar Aiden M 1 Hess 185.99138 free La Crosse-Onalaska, WI-MN 642 A Primera Vista 86\n",
      "Travie McCoy Aiden M 1 Hess 211.12118 free La Crosse-Onalaska, WI-MN 687 Billionaire [feat. Bruno Mars]  (Explicit Album Version) 86\n",
      "Sally Shapiro Aiden M 2 Hess 392.82893 free La Crosse-Onalaska, WI-MN 710 Miracle (Extended Mix) 86\n",
      "Selena Gomez & The Scene Aiden M 1 Hess 172.61669 free La Crosse-Onalaska, WI-MN 760 Stop & Erase 86\n",
      "Damian Marley / Bunny Wailer Aiden M 3 Hess 329.06404 free La Crosse-Onalaska, WI-MN 869 Confrontation 86\n",
      "Feist Aiden M 0 Hess 184.94649 free La Crosse-Onalaska, WI-MN 889 My Moon My Man 86\n",
      "Lewis Black Aiden M 2 Hess 111.80363 free La Crosse-Onalaska, WI-MN 892 Gamblers 86\n",
      "Metric Aiden M 5 Hess 252.99546 free La Crosse-Onalaska, WI-MN 906 Stadium Love 86\n",
      "Natalie Walker Aiden M 2 Hess 191.84281 free La Crosse-Onalaska, WI-MN 917 Colorblind 86\n",
      "Showbread Aiden M 4 Hess 297.58649 free La Crosse-Onalaska, WI-MN 978 The Pig 86\n",
      "Fattburger Aiden M 5 Hess 217.20771 free La Crosse-Onalaska, WI-MN 986 Groovin' 86\n",
      "Metallica Shakira F 1 Hunt 515.21261 free Washington-Arlington-Alexandria, DC-VA-MD-WV 277 Master Of Puppets 84\n",
      "The Moody Blues Shakira F 4 Hunt 258.71628 free Washington-Arlington-Alexandria, DC-VA-MD-WV 83 Tuesday Afternoon 84\n",
      "Soda Stereo Aiden M 8 Ramirez 186.69669 paid New York-Newark-Jersey City, NY-NJ-PA 19 ObservÃÂ¡ndonos (SatÃÂ©lites) 20\n",
      "Survivor Jayden M 0 Fox 245.36771 free New Orleans-Metairie, LA 100 Eye Of The Tiger 101\n",
      "June Carter Cash;Johnny Cash Jayden M 3 Fox 180.21832 free New Orleans-Metairie, LA 1016 If I Were A Carpenter 101\n",
      "12 Stones Jayden M 5 Fox 184.0322 free New Orleans-Metairie, LA 1060 Anthem For The Underdog 101\n",
      "N.E.R.D. FEATURING MALICE Jayden M 0 Fox 288.9922 free New Orleans-Metairie, LA 184 Am I High (Feat. Malice) 101\n",
      "Habib KoitÃÂ© Jayden M 0 Fox 285.1522 free New Orleans-Metairie, LA 185 Din Din Wo 101\n",
      "Black Eyed Peas Jayden M 1 Fox 214.93506 free New Orleans-Metairie, LA 219 Pump It 101\n",
      "Miguel Calo Jayden M 5 Fox 171.17995 free New Orleans-Metairie, LA 282 El Cuatrero 101\n",
      "Crystal Jayden M 1 Fox 251.97669 free New Orleans-Metairie, LA 296 VÃÂNDORMADÃÂR 101\n",
      "Soul II Soul Jayden M 0 Fox 239.67302 free New Orleans-Metairie, LA 298 How Long 101\n",
      "Galactic Jayden M 1 Fox 128.39138 free New Orleans-Metairie, LA 383 Wild Man (featuring Big Chief Bo Dollis) 101\n",
      "Peter Sellers Jayden M 1 Fox 142.39302 free New Orleans-Metairie, LA 408 She Loves You 101\n",
      "Keith Sweat Jayden M 0 Fox 312.99873 free New Orleans-Metairie, LA 419 In Your Eyes (LP Version) 101\n",
      "Dwight Yoakam Jayden M 7 Fox 239.3073 free New Orleans-Metairie, LA 603 You're The One 101\n",
      "Jonas Brothers Jayden M 2 Fox 201.27302 free New Orleans-Metairie, LA 604 Games 101\n",
      "Something Happens Jayden M 5 Fox 175.67302 free New Orleans-Metairie, LA 647 Inviral Love 101\n",
      "Rihanna Jayden M 0 Fox 208.40444 free New Orleans-Metairie, LA 665 Te Amo 101\n",
      "John Mayer Jayden M 0 Fox 259.16036 free New Orleans-Metairie, LA 673 Wait Until Tomorrow 101\n",
      "Neneh Cherry Jayden M 1 Fox 232.202 free New Orleans-Metairie, LA 790 Manchild 101\n",
      "Tommy Emmanuel Jayden M 3 Fox 168.14975 free New Orleans-Metairie, LA 838 Windy & Warm 101\n",
      "Mobin Master Feat. Robin S. Jayden M 1 Fox 428.64281 free New Orleans-Metairie, LA 842 Show Me Love 101\n",
      "Quantic Jayden M 1 Fox 240.22159 free New Orleans-Metairie, LA 936 Sound Of Everything Feat. Alice Russell 101\n",
      "Kalle Baah Jayden M 1 Fox 273.76281 free New Orleans-Metairie, LA 955 Ugly Girls 101\n",
      "Blue October Ayleen F 0 Wise 325.90322 free Columbia, SC 212 Come In Closer 71\n",
      "OutKast Ayleen F 3 Wise 227.52608 free Columbia, SC 70 Ova Da Wudz 71\n",
      "Bright Eyes Ayla F 0 Johnson 279.09179 free Santa Rosa, CA 135 Kathy with a K's Song 63\n",
      "Amy Winehouse Ayla F 0 Johnson 231.52281 free Santa Rosa, CA 223 Stronger Than Me 63\n",
      "Calle 13 Ayla F 2 Johnson 228.72771 free Santa Rosa, CA 330 Sin Coro 63\n",
      "Immortal Technique Ayla F 1 Johnson 171.12771 free Santa Rosa, CA 520 Bin Laden Feat. MosDef (Inst) 63\n",
      "James Blunt Ayla F 1 Johnson 257.95873 free Santa Rosa, CA 544 Goodbye My Lover 63\n",
      "Nine Inch Nails Ayla F 0 Johnson 300.64281 free Santa Rosa, CA 588 Terrible Lie 63\n",
      "Kid Cudi / Ratatat Ayla F 1 Johnson 246.72608 free Santa Rosa, CA 627 Alive (nightmare) 63\n",
      "OneRepublic Ayla F 3 Johnson 208.14322 free Santa Rosa, CA 729 Apologize 63\n",
      "Hooligans Ayla F 0 Johnson 189.98812 free Santa Rosa, CA 785 Szex & KV 63\n",
      "Aaliyah Ayla F 2 Johnson 228.28363 free Santa Rosa, CA 839 More Than A Woman 63\n",
      "Floyd Cramer Ayla F 1 Johnson 202.68363 free Santa Rosa, CA 851 Have I Told You Lately 63\n",
      "Relient K Ayla F 0 Johnson 240.92689 free Santa Rosa, CA 949 I So Hate Consequences (Album Version) 63\n",
      "Alanis Morissette Stefany F 0 White 175.82975 free Lubbock, TX 186 Right Through You (LP Version) 83\n",
      "Newton Faulkner Stefany F 1 White 177.3971 free Lubbock, TX 258 I Need Something 83\n",
      "The Champs Stefany F 7 White 132.0224 free Lubbock, TX 375 Tequila 83\n",
      "Richard Ashcroft Stefany F 1 White 236.35546 free Lubbock, TX 380 Break The Night With Colour 83\n",
      "Mike Jones Stefany F 0 White 240.90077 free Lubbock, TX 533 Mr. Jones (Explicit Version) 83\n",
      "Lonnie Gordon Stefany F 4 White 181.21098 free Lubbock, TX 649 Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 83\n",
      "Rokia TraorÃÂ© Stefany F 0 White 274.88608 free Lubbock, TX 693 Zen 83\n",
      "Summer Cats Stefany F 2 White 219.42812 free Lubbock, TX 697 Wild Rice 83\n",
      "Cherise Stefany F 0 White 229.69424 free Lubbock, TX 772 No Good 4 You 83\n",
      "Tracy Gang Pussy Stefany F 2 White 221.33506 free Lubbock, TX 82 I Have A Wish 83\n",
      "Ticanaf Stefany F 2 White 2594.87302 free Lubbock, TX 867 The Thousand Names of Lord Shiva (Part 1) 83\n",
      "Shakira Jizelle F 6 Benjamin 148.1922 free Plymouth, IN 126 Pienso En Ti 2\n",
      "Los Del Rio Jizelle F 1 Benjamin 249.49506 free Plymouth, IN 323 Macarena 2\n",
      "The Ramones Jizelle F 3 Benjamin 210.18077 free Plymouth, IN 354 Pet Semetary 2\n",
      "King Stitt Sean F 1 Wilson 176.27383 free Philadelphia-Camden-Wilmington, PA-NJ-DE-MD 21 The Ugly One (AKA Lee Van Cleef) 22\n",
      "BeyoncÃÂ© Austin M 0 Rosales 374.59546 free New York-Newark-Jersey City, NY-NJ-PA 1078 Get Me Bodied 12\n",
      "Justin Bieber Austin M 1 Rosales 214.67383 free New York-Newark-Jersey City, NY-NJ-PA 209 One Time 12\n",
      "Slim Dusty Austin M 0 Rosales 198.922 free New York-Newark-Jersey City, NY-NJ-PA 300 Long Black Road 12\n",
      "Kid Cudi Austin M 6 Rosales 227.34322 free New York-Newark-Jersey City, NY-NJ-PA 371 Up Up & Away 12\n",
      "Sara Bareilles Austin M 1 Rosales 260.8322 free New York-Newark-Jersey City, NY-NJ-PA 609 Love Song 12\n",
      "Adam Lambert Austin M 0 Rosales 227.39546 free New York-Newark-Jersey City, NY-NJ-PA 967 Whataya Want From Me 12\n",
      "Dwight Yoakam Morris M 2 Gilmore 239.3073 free Raleigh, NC 177 You're The One 23\n",
      "'N Sync/Phil Collins Morris M 1 Gilmore 143.64689 free Raleigh, NC 351 Trashin' The Camp (Phil And 'N Sync Version) 23\n",
      "Eminem Morris M 0 Gilmore 248.58077 free Raleigh, NC 841 Just Lose It 23\n",
      "Madeleine Peyroux Layla F 19 Griffin 206.18404 paid Lake Havasu City-Kingman, AZ 1051 You're Gonna Make Me Lonesome When You Go 24\n",
      "Max Richter Layla F 33 Griffin 193.38404 paid Lake Havasu City-Kingman, AZ 23 Organum 24\n",
      "Feist Layla F 0 Griffin 184.94649 paid Lake Havasu City-Kingman, AZ 395 My Moon My Man 24\n",
      "Belanova Layla F 23 Griffin 205.50485 paid Lake Havasu City-Kingman, AZ 407 Escena Final 24\n",
      "Less Than Jake Layla F 106 Griffin 154.48771 paid Lake Havasu City-Kingman, AZ 672 Scott Farcas Takes It On The Chin 24\n",
      "The Hippos Layla F 13 Griffin 127.58159 paid Lake Havasu City-Kingman, AZ 723 Freeze Up 24\n",
      "BjÃÂ¶rk Layla F 8 Griffin 348.57751 paid Lake Havasu City-Kingman, AZ 739 Undo 24\n",
      "Shadows Fall Layla F 9 Griffin 220.3424 paid Lake Havasu City-Kingman, AZ 809 War 24\n",
      "Gabin featuring China Moses Layla F 11 Griffin 249.36444 paid Lake Havasu City-Kingman, AZ 879 The Other Way Round (Feat. China Moses) 24\n",
      "Insane Clown Posse Layla F 117 Griffin 233.82159 paid Lake Havasu City-Kingman, AZ 984 My Axe 24\n",
      "Skillet Bronson M 7 Harris 233.32526 free Eugene, OR 1026 Rebirthing (Album Version) 33\n",
      "NEEDTOBREATHE Bronson M 2 Harris 225.82812 free Eugene, OR 134 Don't Wait For Daylight (Album Version) 33\n",
      "Florence + The Machine Bronson M 0 Harris 219.66322 free Eugene, OR 251 Dog Days Are Over (Radio Edit) 33\n",
      "Underworld Bronson M 1 Harris 329.53424 free Eugene, OR 314 Teardrop 33\n",
      "Silverchair Bronson M 0 Harris 257.72363 free Eugene, OR 358 Straight Lines 33\n",
      "Train Bronson M 1 Harris 239.04608 free Eugene, OR 399 If It's Love 33\n",
      "Moby Bronson M 0 Harris 240.74404 free Eugene, OR 516 In This World 33\n",
      "MGMT Bronson M 1 Harris 264.17587 free Eugene, OR 682 Time To Pretend 33\n",
      "Reel Big Fish Bronson M 2 Harris 210.65098 free Eugene, OR 827 Beer 33\n",
      "Catupecu Machu Bronson M 3 Harris 191.79057 free Eugene, OR 937 Y Lo Que Quiero Es Que Pises Sin El Suelo 33\n",
      "Iron Maiden Bronson M 1 Harris 409.5473 free Eugene, OR 941 Ghost Of The Navigator 33\n",
      "Leonard Cohen Sara F 63 Johnson 298.57914 paid Winston-Salem, NC 152 The Stranger Song 95\n",
      "Britt Nicole Sara F 31 Johnson 232.85506 paid Winston-Salem, NC 222 Like A Star 95\n",
      "Tub Ring Sara F 6 Johnson 233.69098 paid Winston-Salem, NC 276 Invalid 95\n",
      "Modwheelmood Sara F 84 Johnson 209.71057 paid Winston-Salem, NC 411 Sunday Morning 95\n",
      "Plain White T's Sara F 43 Johnson 232.202 paid Winston-Salem, NC 564 Hey There Delilah 95\n",
      "Carrie Underwood Gianna F 2 Jones 195.70893 free New York-Newark-Jersey City, NY-NJ-PA 107 Look At Me 38\n",
      "Maximilian Hecker Jacqueline F 1 Lynch 295.60118 free Atlanta-Sandy Springs-Roswell, GA 28 Let Me Out 29\n",
      "John Mayer Jacqueline F 2 Lynch 267.38893 free Atlanta-Sandy Springs-Roswell, GA 295 Why Georgia 29\n",
      "Metallica Jacqueline F 5 Lynch 515.21261 paid Atlanta-Sandy Springs-Roswell, GA 353 Master Of Puppets 29\n",
      "The Smiths Jacqueline F 18 Lynch 196.67546 paid Atlanta-Sandy Springs-Roswell, GA 372 The Boy With The Thorn In His Side 29\n",
      "Anjulie Jacqueline F 6 Lynch 194.63791 paid Atlanta-Sandy Springs-Roswell, GA 389 Boom 29\n",
      "Jake Hess Jacqueline F 73 Lynch 199.26159 paid Atlanta-Sandy Springs-Roswell, GA 486 You And Me Jesus 29\n",
      "FM Static Jacqueline F 36 Lynch 182.41261 paid Atlanta-Sandy Springs-Roswell, GA 556 Boy Meets Girl (And Vice Versa) 29\n",
      "Scritti Politti Jacqueline F 59 Lynch 286.9024 paid Atlanta-Sandy Springs-Roswell, GA 559 Wood Beez 29\n",
      "Weezer Jacqueline F 126 Lynch 139.65016 paid Atlanta-Sandy Springs-Roswell, GA 589 Photograph 29\n",
      "The Presets Jacqueline F 6 Lynch 271.85587 paid Atlanta-Sandy Springs-Roswell, GA 709 Yippiyo-Ay 29\n",
      "Train Jacqueline F 17 Lynch 205.45261 paid Atlanta-Sandy Springs-Roswell, GA 898 Marry Me 29\n",
      "Trafik Jacqueline F 6 Lynch 380.21179 paid Atlanta-Sandy Springs-Roswell, GA 924 Dirty Word 29\n",
      "Rage Against The Machine Jacqueline F 12 Lynch 338.46812 paid Atlanta-Sandy Springs-Roswell, GA 963 The Ghost Of Tom Joad 29\n",
      "Hellhammer Jordan F 1 Hicks 241.71057 free Salinas, CA 1000 Maniac 37\n",
      "Alex Band Jordan F 0 Hicks 238.8371 free Salinas, CA 145 Tonight (Single Mix) 37\n",
      "Hot Hot Heat Jordan F 4 Hicks 190.01424 free Salinas, CA 240 Soldier In A Box (Album Version) 37\n",
      "Chromeo Jordan F 6 Hicks 348.65587 free Salinas, CA 304 You're So Gangsta 37\n",
      "Shinedown Jordan F 0 Hicks 233.97832 free Salinas, CA 449 Sound Of Madness (Album Version) 37\n",
      "The Be Good Tanyas Jordan F 4 Hicks 225.04444 free Salinas, CA 499 A Thousand Tiny Pieces 37\n",
      "The Unicorns Jordan F 3 Hicks 222.74567 free Salinas, CA 547 Sea Ghost 37\n",
      "Elliott Smith Jordan F 0 Hicks 141.42649 free Salinas, CA 550 Between The Bars 37\n",
      "Anna Tatangelo Jordan F 0 Hicks 218.93179 free Salinas, CA 552 Il Mio Amico 37\n",
      "OneRepublic Jordan F 1 Hicks 212.32281 free Salinas, CA 581 Goodbye_ Apathy 37\n",
      "Global Deejays Feat. Rozalla Jordan F 0 Hicks 398.52363 free Salinas, CA 684 EverybodyÃÂ´s Free 37\n",
      "Veracocha Jordan F 0 Hicks 506.93179 free Salinas, CA 715 Carte Blanche 37\n",
      "Creedence Clearwater Revival Jordan F 2 Hicks 184.73751 free Salinas, CA 814 Run Through The Jungle 37\n",
      "C.J. Lewis Jordan F 1 Hicks 201.27302 free Salinas, CA 935 Sweets For My Sweet 37\n",
      "Octopus Project Magdalene F 1 Herman 498.33751 free Dallas-Fort Worth-Arlington, TX 141 Hypnopaedia 77\n",
      "Booka Shade Magdalene F 1 Herman 142.10567 free Dallas-Fort Worth-Arlington, TX 244 No Difference 77\n",
      "Simon Harris Magdalene F 1 Herman 195.83955 free Dallas-Fort Worth-Arlington, TX 656 Sample Track 2 77\n",
      "Kid Cudi / MGMT / Ratatat Magdalene F 5 Herman 295.67955 free Dallas-Fort Worth-Arlington, TX 707 Pursuit Of Happiness (nightmare) 77\n",
      "Unkle Bob Mohammad M 10 Rodriguez 160.10404 paid Sacramento--Roseville--Arden-Arcade, CA 1075 One By One 88\n",
      "Ashlee Simpson Mohammad M 5 Rodriguez 214.43873 paid Sacramento--Roseville--Arden-Arcade, CA 1097 Autobiography 88\n",
      "Leonard Cohen Mohammad M 5 Rodriguez 267.67628 free Sacramento--Roseville--Arden-Arcade, CA 193 I'm Your Man 88\n",
      "Katy Perry Mohammad M 1 Rodriguez 179.40853 free Sacramento--Roseville--Arden-Arcade, CA 205 I Kissed A Girl 88\n",
      "Steely Dan Mohammad M 0 Rodriguez 457.89995 free Sacramento--Roseville--Arden-Arcade, CA 206 Deacon Blues 88\n",
      "Delirious? Mohammad M 2 Rodriguez 397.26975 free Sacramento--Roseville--Arden-Arcade, CA 312 History Maker 88\n",
      "The Devil Wears Prada Mohammad M 1 Rodriguez 205.66159 free Sacramento--Roseville--Arden-Arcade, CA 325 Lord Xenu 88\n",
      "Nate Dogg Mohammad M 2 Rodriguez 356.38812 free Sacramento--Roseville--Arden-Arcade, CA 441 Never Leave Me Alone 88\n",
      "Europe Mohammad M 1 Rodriguez 346.69669 free Sacramento--Roseville--Arden-Arcade, CA 471 The Final Countdown 88\n",
      "That Petrol Emotion Mohammad M 2 Rodriguez 265.1424 free Sacramento--Roseville--Arden-Arcade, CA 565 Sooner Or Later 88\n",
      "Modest Mouse Mohammad M 3 Rodriguez 259.47383 free Sacramento--Roseville--Arden-Arcade, CA 591 The Good Times Are Killing Me 88\n",
      "Crosby_ Stills & Nash Mohammad M 3 Rodriguez 329.19465 free Sacramento--Roseville--Arden-Arcade, CA 614 Wooden Ships (LP Version) 88\n",
      "Eminem Mohammad M 0 Rodriguez 252.73424 free Sacramento--Roseville--Arden-Arcade, CA 659 Still Don't Give A Fuck 88\n",
      "Calle 13 Mohammad M 1 Rodriguez 239.3073 free Sacramento--Roseville--Arden-Arcade, CA 679 AtrÃÂ©vete te te 88\n",
      "Hot Chip Mohammad M 0 Rodriguez 283.32363 free Sacramento--Roseville--Arden-Arcade, CA 688 We're Looking For A Lot Of Love 88\n",
      "Blind Melon Mohammad M 9 Rodriguez 166.39955 paid Sacramento--Roseville--Arden-Arcade, CA 726 Walk 88\n",
      "Telex Mohammad M 46 Rodriguez 44.48608 paid Sacramento--Roseville--Arden-Arcade, CA 744 ClichÃÂ© 88\n",
      "Kings Of Leon Mohammad M 38 Rodriguez 237.26975 paid Sacramento--Roseville--Arden-Arcade, CA 812 Trunk 88\n",
      "Pennywise Mohammad M 0 Rodriguez 188.26404 free Sacramento--Roseville--Arden-Arcade, CA 87 The Western World 88\n",
      "Smash Mouth Mohammad M 59 Rodriguez 216.39791 paid Sacramento--Roseville--Arden-Arcade, CA 888 All Star 88\n",
      "Rammstein Mohammad M 55 Rodriguez 188.57751 paid Sacramento--Roseville--Arden-Arcade, CA 900 Feuer Frei 88\n",
      "Building 429 Mohammad M 2 Rodriguez 300.61669 paid Sacramento--Roseville--Arden-Arcade, CA 961 Majesty (LP Version) 88\n",
      "Rilo Kiley Mohammad M 35 Rodriguez 322.42893 paid Sacramento--Roseville--Arden-Arcade, CA 969 A Man/Me/Then Jim (Album Version) 88\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Dwight Yoakam Mohammad M 8 Rodriguez 227.05587 paid Sacramento--Roseville--Arden-Arcade, CA 999 Listen (LP Version) 88\n",
      "B.o.B Tegan F 15 Levine 269.63546 paid Portland-South Portland, ME 1004 Nothin' On You [feat. Bruno Mars] (Album Version) 80\n",
      "311 Tegan F 40 Levine 113.26649 paid Portland-South Portland, ME 1065 Loco 80\n",
      "Washed Out Tegan F 3 Levine 168.6722 free Portland-South Portland, ME 165 New Theory 80\n",
      "The Avett Brothers Tegan F 3 Levine 307.9571 free Portland-South Portland, ME 194 All My Mistakes (US4BW0700133) 80\n",
      "Ottmar Liebert Tegan F 4 Levine 174.54975 free Portland-South Portland, ME 243 Festival (Of 7 Lights) 80\n",
      "School Of Seven Bells Tegan F 0 Levine 262.37342 free Portland-South Portland, ME 254 Chain 80\n",
      "Paul Stanley Tegan F 1 Levine 187.32363 free Portland-South Portland, ME 279 Live To Win 80\n",
      "Roy Ayers Tegan F 1 Levine 240.90077 free Portland-South Portland, ME 307 Searching 80\n",
      "Arctic Monkeys Tegan F 1 Levine 152.60689 free Portland-South Portland, ME 329 Baby I'm Yours 80\n",
      "Bright Eyes Tegan F 53 Levine 375.95383 paid Portland-South Portland, ME 342 I Must Belong Somewhere 80\n",
      "Death In Vegas Tegan F 9 Levine 270.23628 paid Portland-South Portland, ME 376 Girls 80\n",
      "Stevie Ray Vaughan And Double Trouble Tegan F 1 Levine 258.82077 paid Portland-South Portland, ME 398 Life Without You 80\n",
      "isis Tegan F 14 Levine 406.12526 paid Portland-South Portland, ME 409 Carry 80\n",
      "Eminem Tegan F 49 Levine 212.00934 paid Portland-South Portland, ME 416 I'm Shady 80\n",
      "LCD Soundsystem Tegan F 3 Levine 221.64853 paid Portland-South Portland, ME 434 Drunk Girls 80\n",
      "Katy Perry Tegan F 14 Levine 246.41261 paid Portland-South Portland, ME 435 Thinking Of You 80\n",
      "Deepest Blue Tegan F 16 Levine 210.54649 paid Portland-South Portland, ME 481 Deepest Blue 80\n",
      "Black Moth Super Rainbow Tegan F 4 Levine 221.28281 paid Portland-South Portland, ME 545 Early 70's Gymnastics 80\n",
      "Green Day Tegan F 81 Levine 185.12934 paid Portland-South Portland, ME 548 American Idiot [feat. Green Day & The Cast Of American Idiot] (Album Version) 80\n",
      "JoÃÂ£o Gilberto Tegan F 54 Levine 258.37669 paid Portland-South Portland, ME 574 Ave Maria No Morro 80\n",
      "The Killers Tegan F 10 Levine 360.75057 paid Portland-South Portland, ME 602 Human 80\n",
      "Mariah Carey / Twista Tegan F 42 Levine 194.7424 paid Portland-South Portland, ME 611 One And Only 80\n",
      "Shinedown Tegan F 59 Levine 228.5971 paid Portland-South Portland, ME 620 Begin Again (Album Version) 80\n",
      "Tom Petty And The Heartbreakers Tegan F 5 Levine 173.322 paid Portland-South Portland, ME 666 Baby's A Rock 'N' Roller (Album Version) 80\n",
      "Richard Holmes Tegan F 40 Levine 213.99465 paid Portland-South Portland, ME 774 Flyjack 80\n",
      "the bird and the bee Tegan F 2 Levine 198.1122 paid Portland-South Portland, ME 823 Last Day Of Our Love 80\n",
      "Johnny Winter Tegan F 26 Levine 223.81669 paid Portland-South Portland, ME 848 Still Alive And Well 80\n",
      "After 7 Tegan F 21 Levine 241.94567 paid Portland-South Portland, ME 893 Nights Like This 80\n",
      "Deadmau5 & Kaskade Tegan F 24 Levine 595.56526 paid Portland-South Portland, ME 903 I Remember 80\n",
      "Train Tegan F 28 Levine 216.76363 paid Portland-South Portland, ME 910 Hey_ Soul Sister 80\n",
      "Go West Tegan F 5 Levine 259.49995 paid Portland-South Portland, ME 915 Never Let Them See You Sweat 80\n",
      "Die Mooskirchner Tegan F 14 Levine 169.29914 paid Portland-South Portland, ME 933 Frisch und g'sund 80\n",
      "Erik Hassle Tegan F 70 Levine 183.43138 paid Portland-South Portland, ME 992 Hurtful 80\n",
      "Rihanna Christian F 1 Porter 293.82485 free Elkhart-Goshen, IN 10 Rehab 11\n",
      "Shakira Christian F 0 Porter 193.82812 free Elkhart-Goshen, IN 366 Inevitable 11\n",
      "Paramore Chloe F 1 Roth 267.65016 free Indianapolis-Carmel-Anderson, IN 1059 The Only Exception (Album Version) 78\n",
      "Dwight Yoakam Chloe F 5 Roth 239.3073 free Indianapolis-Carmel-Anderson, IN 176 You're The One 78\n",
      "Rancid Chloe F 2 Roth 152.76363 free Indianapolis-Carmel-Anderson, IN 247 Hooligans (Album Version) 78\n",
      "Taylor Swift Chloe F 1 Roth 233.89995 free Indianapolis-Carmel-Anderson, IN 507 Love Story 78\n",
      "Pixies Chloe F 0 Roth 113.78893 free Indianapolis-Carmel-Anderson, IN 511 Break My Body 78\n",
      "Jason Mraz & Colbie Caillat Chloe F 0 Roth 189.6224 free Indianapolis-Carmel-Anderson, IN 704 Lucky (Album Version) 78\n",
      "Parov Stelar Chloe F 1 Roth 203.65016 free Indianapolis-Carmel-Anderson, IN 925 Good Bye Emily (feat. Gabriella Hanninen) 78\n",
      "Cute Is What We Aim For Chloe F 0 Roth 222.22322 free Indianapolis-Carmel-Anderson, IN 931 Risque (Album Version) 78\n",
      "The Casualties Cierra F 3 Finley 181.49832 free Richmond, VA 132 Punx Unite 96\n",
      "Charttraxx Karaoke Cierra F 4 Finley 225.17506 free Richmond, VA 443 Fireflies 96\n",
      "The Academy Is... Cierra F 2 Finley 207.96036 free Richmond, VA 473 We've Got A Big Mess On Our Hands (Album Edit) 96\n",
      "The Agony Scene Cierra F 3 Finley 175.85587 free Richmond, VA 810 Screams Turn To Silence (Album Version) 96\n",
      "Skillet Kevin M 0 Arellano 178.02404 free Harrisburg-Carlisle, PA 153 Monster (Album Version) 66\n",
      "Jason Mraz & Colbie Caillat Kevin M 0 Arellano 189.6224 free Harrisburg-Carlisle, PA 187 Lucky (Album Version) 66\n",
      "Kings Of Leon Kevin M 2 Arellano 231.81016 free Harrisburg-Carlisle, PA 191 Use Somebody 66\n",
      "Lily Allen Kevin M 2 Arellano 194.76853 free Harrisburg-Carlisle, PA 319 Smile 66\n",
      "Cheb Mami - K-Maro Kevin M 0 Arellano 191.73832 free Harrisburg-Carlisle, PA 370 Nos Couleurs (Feat K Maro) 66\n",
      "Juanes Kevin M 1 Arellano 247.37914 free Harrisburg-Carlisle, PA 387 Damelo 66\n",
      "Guns N' Roses Kevin M 7 Arellano 406.17751 free Harrisburg-Carlisle, PA 450 Paradise City 66\n",
      "Evanescence Kevin M 3 Arellano 263.78404 free Harrisburg-Carlisle, PA 501 My Immortal (Album Version) 66\n",
      "Fu Kevin M 1 Arellano 280.05832 free Harrisburg-Carlisle, PA 514 Ja I Ty 66\n",
      "Eric B. & Rakim Kevin M 1 Arellano 263.91465 free Harrisburg-Carlisle, PA 525 Move The Crowd 66\n",
      "MGMT Kevin M 1 Arellano 264.17587 free Harrisburg-Carlisle, PA 541 Time To Pretend 66\n",
      "Marc Et Claude Kevin M 0 Arellano 220.86485 free Harrisburg-Carlisle, PA 549 I Need Your Lovin' (Like The Sunshine) (Radio Edit) 66\n",
      "Lou Gramm Kevin M 3 Arellano 296.17587 free Harrisburg-Carlisle, PA 676 Just Between You And Me (Remastered LP Version) 66\n",
      "Rick James Kevin M 1 Arellano 263.60118 free Harrisburg-Carlisle, PA 714 Ghetto Life 66\n",
      "Smash Mouth Kevin M 10 Arellano 216.39791 free Harrisburg-Carlisle, PA 815 All Star 66\n",
      "Viola Kevin M 2 Arellano 260.25751 free Harrisburg-Carlisle, PA 850 Nostalgia Amnesia 66\n",
      "Alice In Chains Kevin M 0 Arellano 239.35955 free Harrisburg-Carlisle, PA 934 Take Her Out 66\n",
      "JosÃÂ© Alfredo Jimenez Con Mariachi Vargas De Tecalitlan Brayden M 9 Clark 129.27955 free New York-Newark-Jersey City, NY-NJ-PA 477 El Rey 41\n",
      "VCR Brayden M 0 Clark 193.69751 free New York-Newark-Jersey City, NY-NJ-PA 736 On Its Way Out 41\n",
      "Jack Johnson Kaleb M 1 Cook 173.42649 free Yuba City, CA 213 Wrong Turn 54\n",
      "Johnny Bristol Kaleb M 1 Cook 198.1122 free Yuba City, CA 364 Hang On In There Baby 54\n",
      "Bloodhound Gang Kaleb M 0 Cook 260.20526 free Yuba City, CA 53 Uhn Tiss Uhn Tiss Uhn Tiss 54\n",
      "Plain White T's Kaleb M 8 Cook 232.202 free Yuba City, CA 616 Hey There Delilah 54\n",
      "Coldplay Kaleb M 2 Cook 298.762 free Yuba City, CA 683 Shiver 54\n",
      "Ozzy Osbourne Kaleb M 1 Cook 248.42404 free Yuba City, CA 743 Secret Loser 54\n",
      "Evanescence Kaleb M 1 Cook 279.27465 free Yuba City, CA 770 Tourniquet 54\n",
      "Maya Nasri Kaleb M 1 Cook 245.75955 free Yuba City, CA 822 Wayli Aah 54\n",
      "Kim Carnes Kaleb M 4 Cook 217.28608 free Yuba City, CA 951 Crazy In The Night (Barking At Airplanes) 54\n",
      "Train Carlos M 0 Carter 216.76363 free Miami-Fort Lauderdale-West Palm Beach, FL 125 Hey_ Soul Sister 27\n",
      "Sean Kingston and Justin Bieber Carlos M 1 Carter 201.9522 free Miami-Fort Lauderdale-West Palm Beach, FL 292 Eenie Meenie 27\n"
     ]
    }
   ],
   "source": [
    "query = \"SELECT * FROM music_history\"\n",
    "try:\n",
    "    rows = session.execute(query)\n",
    "except Exception as e:\n",
    "    print(e)\n",
    "    \n",
    "for row in rows:\n",
    "    print (row.artist, row.first_name, row.gender, row.item_in_session, row.last_name, row.length, row.level, row.location, row.session_id, row.song, row.user_id)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "View 'music_history_by_item_view' has been created successfully.\n"
     ]
    }
   ],
   "source": [
    "query = \"\"\"\n",
    "CREATE MATERIALIZED VIEW music_history_by_item_view AS\n",
    "SELECT *\n",
    "FROM music_history\n",
    "WHERE user_id IS NOT NULL AND session_id IS NOT NULL AND item_in_session IS NOT NULL\n",
    "PRIMARY KEY ((user_id, session_id), item_in_session);\n",
    "\"\"\"\n",
    "try:\n",
    "    session.execute(query)\n",
    "    print(\"View 'music_history_by_item_view' has been created successfully.\")\n",
    "except Exception as e:\n",
    "    print(f\"An error occurred while trying to create the table: {e}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 16,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz\n"
     ]
    }
   ],
   "source": [
    "## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\\\n",
    "## for userid = 10, sessionid = 182\n",
    "query = \"SELECT * FROM music_history_by_item_view WHERE user_id='10' AND session_id='182' ORDER BY item_in_session desc\"\n",
    "try:\n",
    "    rows = session.execute(query)\n",
    "except Exception as e:\n",
    "    print(e)\n",
    "    \n",
    "for row in rows:\n",
    "    full_name = f\"{row.first_name} {row.last_name}\" \n",
    "    print(row.artist, row.song, full_name)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 35,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "View 'music_history_by_song_view' has been created successfully.\n"
     ]
    }
   ],
   "source": [
    "query = \"\"\"\n",
    "CREATE MATERIALIZED VIEW music_history_by_song_view AS\n",
    "SELECT song, user_id, session_id, first_name, last_name\n",
    "FROM music_history\n",
    "WHERE song IS NOT NULL AND user_id IS NOT NULL AND session_id IS NOT NULL\n",
    "PRIMARY KEY (song, user_id, session_id);\n",
    "\"\"\"\n",
    "try:\n",
    "    session.execute(query)\n",
    "    print(\"View 'music_history_by_song_view' has been created successfully.\")\n",
    "except Exception as e:\n",
    "    print(f\"An error occurred while trying to create the table: {e}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 43,
   "metadata": {},
   "outputs": [],
   "source": [
    "## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'\n",
    "\n",
    "query = \"SELECT first_name, last_name FROM music_history_by_song_view WHERE song='All Hands Against His Own' \"\n",
    "\n",
    "try:\n",
    "    rows = session.execute(query)\n",
    "    users = set()  \n",
    "    for row in rows:\n",
    "        users.add((row.first_name, row.last_name))\n",
    "    for user in users:\n",
    "        print(f\"{user[0]} {user[1]}\")\n",
    "except Exception as e:\n",
    "    print(f\"An error occurred: {e}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Drop the tables before closing out the sessions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "An error occurred while trying to drop the table: name 'session' is not defined\n"
     ]
    }
   ],
   "source": [
    "## TO-DO: Drop the table before closing out the sessions\n",
    "query = \"DROP TABLE IF EXISTS music_history\"\n",
    "\n",
    "\n",
    "try:\n",
    "    session.execute(query)\n",
    "    print(\"Table 'music_history' has been dropped successfully.\")\n",
    "except Exception as e:\n",
    "    print(f\"An error occurred while trying to drop the table: {e}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 26,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Table 'music_history_by_session_view' has been dropped successfully.\n"
     ]
    }
   ],
   "source": [
    "## TO-DO: Drop the view before closing out the sessions\n",
    "query = \"DROP MATERIALIZED VIEW music_history_by_session_view\"\n",
    "\n",
    "\n",
    "try:\n",
    "    session.execute(query)\n",
    "    print(\"Table 'music_history_by_session_view' has been dropped successfully.\")\n",
    "except Exception as e:\n",
    "    print(f\"An error occurred while trying to drop the table: {e}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 44,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Table 'music_history_by_item_view' has been dropped successfully.\n"
     ]
    }
   ],
   "source": [
    "## TO-DO: Drop the view before closing out the sessions\n",
    "query = \"DROP MATERIALIZED VIEW music_history_by_item_view\"\n",
    "\n",
    "\n",
    "try:\n",
    "    session.execute(query)\n",
    "    print(\"Table 'music_history_by_item_view' has been dropped successfully.\")\n",
    "except Exception as e:\n",
    "    print(f\"An error occurred while trying to drop the table: {e}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 33,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Table 'music_history_by_song_view' has been dropped successfully.\n"
     ]
    }
   ],
   "source": [
    "## TO-DO: Drop the view before closing out the sessions\n",
    "query = \"DROP MATERIALIZED VIEW music_history_by_song_view\"\n",
    "\n",
    "\n",
    "try:\n",
    "    session.execute(query)\n",
    "    print(\"Table 'music_history_by_song_view' has been dropped successfully.\")\n",
    "except Exception as e:\n",
    "    print(f\"An error occurred while trying to drop the table: {e}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Close the session and cluster connection¶"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "session.shutdown()\n",
    "cluster.shutdown()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}
